# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションにおけるランキングシステムの公平性について検討することを目的としています。競技の公式説明に基づき、評価方法や現在のシステムの設定を理解し、特にTrueSkillレーティングシステムを用いたスコアリングの仕組みを探求します。

### 問題の取り組み
Notebookでは、次の問題に取り組んでいます：
- LLM 20 Questionsコンペティションにおけるレーティングシステムの構造とその公平性についての評価
- 競技が実際にどのように行われ、どのようにレーティングが決まるかを復元するモデルの構築
- 提出物の勝率や引き分けの確率に影響を与える因子の分析

### 解決手法とライブラリ
この問題に対して、以下の手法とライブラリが用いられています：

1. **TrueSkillライブラリ**:
   - TrueSkillを用いて対戦をシミュレーション。具体的には、`trueskill`というPythonライブラリをインストールし、各プレイヤーのスキルを`Rating`クラスで表現します。レーティングを更新するための`rate()`関数を使用。

2. **データ処理と分析**:
   - Pandasを利用して、ゲームデータを収集し、ゲーム結果の解析を行います。引き分けの確率や、プレイヤーの成功率を計算するためのデータフレームを生成します。

3. **シミュレーションクラス**:
   - `Competition`クラスを作成し、プレイヤーのスキルを基に試合を行う環境を構築。このクラスにおいて、複数の競技をシミュレーションし、結果を視覚化します。

4. **可視化**:
   - Matplotlibを使用して、シミュレーションから得た結果やプレイヤーのパフォーマンスをプロットし、視覚的に分析を行います。

### 結論
Notebookでは、LLM 20 Questionsコンペティションのレーティングシステムは、偶然が多く含まれる可能性があるとの結果が示されています。具体的には、スキルに基づくマッチングが正しく機能していないことが指摘され、何らかの形で公平性を損なう要因が存在することが示唆されています。

---


# 用語概説 
以下に、初心者がつまずきやすい専門用語について簡単な解説を示します。

1. **レーティングシステム (Rating System)**:
   - ゲームや競技において、プレイヤーの能力やスキルを数値化して相対的に順位付けする仕組み。TrueSkillやGlickoなどの異なるレーティングシステムが存在する。

2. **TrueSkill**:
   - Microsoftが開発したレーティングシステムで、特に二人対二人の対戦で使用される。プレイヤーのスキルを分布としてモデル化し（ガウス分布）、勝敗に応じてその分布を更新する。

3. **β (ベータ)**:
   - TrueSkillのパラメータで、勝利の確率を調整するための距離を表す。一般的に、これはプレイヤーのスキルの変動に関する不確実性を示す。

4. **τ (タウ)**:
   - TrueSkillのパラメータで、レーティングが安定するための動的要因を制限する。これにより、スキルの変化が急激にならないよう制御される。

5. **σ (シグマ)**:
   - TrueSkillにおけるスキルの不確実性を示すパラメータ。高い値はプレイヤーのスキルが変動するリスクが高いことを示す。

6. **引き分けの確率 (Draw Probability)**:
   - ゲームの結果が引き分けである確率。TrueSkillではこの確率が設定され、ペアリングの公平さを保つために考慮される。

7. **バリデーションエピソード (Validation Episode)**:
   - 新しいボットが正常に動作するか確認するために、自分自身のコピーと対戦して評価するプロセス。

8. **ダミーボット (Dummy Bot)**:
   - 実力が極端に低いボットで、実際の競技には参加していないものである場合もある。通常、他のボットと組み合わせて互いに対戦させる際に使用する。

9. **レーティングの履歴 (Rating History)**:
   - プレイヤーやボットのレーティングの変遷を記録したデータ。これにより時間経過に伴うパフォーマンスの変化を追跡できる。

10. **ランダムサンプリング (Random Sampling)**:
    - 一定の範囲から無作為に選ばれたデータの集まり。ゲームのプレイヤーを選ぶ際、真の能力を反映できるように利用される。

11. **数理モデル (Mathematical Model)**:
    - 現実のシステムや現象を数式や数学的な方程式を用いて表現したもの。ここでは、プレイヤーのスキルをモデル化するために使用されている。

これらの用語は、コンペティションで利用されるシステムやアルゴリズムを理解する上で特に重要であり、初心者がつまずく可能性のある部分です。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

The fairness of the ranking system of the [LLM 20 Questions competition](https://www.kaggle.com/competitions/llm-20-questions/overview) has been questioned multiple times in discussions, so I decided to write a small toying notebook to evaluate it.  We're not going to evaluate the maths, just play with data.

# Official description of the ranking system

At the time of writing (July 17th), the competition's overview states :
> ### Evaluation
>
> Each day your team is able to submit up to 5 agents (bots) to the competition. Each submission will play episodes (games) against other bots on the leaderboard that have a similar skill rating. Over time, skill ratings will go up with wins, down with losses, or evened out with ties.
> 
> This competition is configured to run in a cooperative, 2 vs. 2 format. Your bot will be randomly paired with a bot of similar skill in order to face off against another random pairing. On each pair, one bot will be randomly assigned as questioner and the other as answerer. Since you win/lose/tie as a pair, you are incentivized to work together!
> 
> Every bot submitted will continue to play episodes until the end of the competition, with newer bots selected to play more frequently. Once you have reached three active submissions, older ones will be deactivated. On the leaderboard, only your best scoring bot will be shown, but you can track the progress of all of your submissions on your Submissions page.
> 
> Each submission has an estimated skill rating which is modeled by a Gaussian N(μ,σ2) where μ is the estimated skill and σ represents the uncertainty of that estimate which will decrease over time.
> 
> When you upload a submission, we first play a validation episode where that submission plays against copies of itself to make sure it works properly. If the episode fails, the submission is marked as error and you can download the agent logs to help figure out why. Otherwise, we initialize the submission with μ0=600 and it joins the pool of for ongoing evaluation. At this time we also deactivate older agents if the total number of active agents is greater than three.
> 
> ### Ranking System
> 
> After an episode finishes, we'll update the rating estimate for all bots in the episode. If one bot pair won, we'll increase their μ and decrease the opponent's μ -- if the result was a tie, then we'll move the μ values closer towards their mean. The updates will have magnitude relative to the deviation from the expected result based on the previous μ values, and also relative to each bot’s uncertainty σ. We also reduce the σ terms relative to the amount of information gained by the result. The score by which your bot wins or loses an episode does not affect the skill rating updates.
> 
> ### Final Evaluation
> 
> At the submission deadline on August 13, 2024, submissions will be locked. From August 13, 2024 to August 27th, 2024 we will continue to run episodes against a new set of unpublished, secret words. During this period only your three active submissions will be eligible for the leaderboard. At the conclusion of this period, the leaderboard is final.

# TrueSkill

I don't know much about rating systems.  A quick search shows that a candidate for a system with a $\mu$ **and** $\sigma$ parameters may be the [Glicko rating system](https://en.wikipedia.org/wiki/Glicko_rating_system).  I can't remember how I came to this conclusion (maybe it had to do with team play ?), but another more likely candidate is [TrueSkill](https://en.wikipedia.org/wiki/TrueSkill).  I won't go into details, follow the Wikipedia link if you're interested.  It seems quite popular in the world of video games.  There is a [TrueSkill 2](https://www.microsoft.com/en-us/research/uploads/prod/2018/03/trueskill2.pdf), but at first glance its improvements don't seem relevant for our kind of competition.

## Installation

Fortunately, there is a [Python library](https://github.com/sublee/trueskill) of TrueSkill available on [Pypi](https://pypi.org/project/trueskill/), let's install it :

</div>
<div class="column-right">

# 日本語訳

[LLM 20 Questionsコンペティション](https://www.kaggle.com/competitions/llm-20-questions/overview)のランキングシステムの公平性については、討論の中で何度も疑問が呈されています。そこで、私はその評価を行うための簡単なノートを作成することにしました。数学の評価は行わず、データを扱ってみます。

# ランキングシステムの公式説明

執筆時点（7月17日）で、コンペティションの概要は次のようになっています：
> ### 評価
>
> 各チームは毎日、最大5つのエージェント（ボット）をコンペティションに提出できます。各提出は、リーダーボード上の同等のスキルレーティングを持つ他のボットと対戦するエピソード（ゲーム）をプレイします。時間が経つにつれて、勝利によってスキルレーティングは上昇し、敗北によって低下し、引き分けによって均一化されます。
> 
> このコンペティションは協力的な2対2の形式で運営されます。あなたのボットは、同等のスキルを持つボットとランダムにペアになり、別のランダムなペアと対戦します。各ペアでは、一方のボットが質問者として、もう一方が回答者としてランダムに割り当てられます。ペアとして勝ったり負けたりするため、協力することが奨励されます！
> 
> 提出されたすべてのボットは、コンペティション終了までエピソードをプレイし続け、新しいボットはより頻繁に選ばれます。3つのアクティブな提出に達すると、古いボットは非アクティブになります。リーダーボードには最高得点のボットのみが表示されますが、すべての提出の進行状況は「提出」ページで追跡できます。
> 
> 各提出物には、ガウス分布N(μ,σ2)でモデル化された推定スキルレーティングがあります。μは推定スキルを、σはその推定の不確実性を表し、時間と共に減少します。
> 
> 提出物をアップロードすると、まずその提出物が正常に機能するか確認するためのバリデーションエピソードが行われます。このエピソードが失敗した場合、提出物はエラーとしてマークされ、原因を特定するためにエージェントログをダウンロードできます。そうでない場合は、μ0=600で提出物を初期化し、継続的な評価のためにプールに追加されます。この時点で、アクティブなエージェントの総数が3を超える場合、古いエージェントは非アクティブ化されます。
> 
> ### ランキングシステム
> 
> エピソードが終了すると、そのエピソードに参加したすべてのボットのレーティング推定値が更新されます。あるボットペアが勝利した場合、そのペアのμは増加し、対戦相手のμは減少します。引き分けの場合は、μの値が平均値に近づくように調整されます。更新の大きさは、以前のμ値に基づく予想結果からの偏差及び各ボットの不確実性σに比例します。また、結果によって得られた情報量に応じてσの項も減少させます。エピソードでの勝利または敗北によるボットのスコアは、スキルレーティングの更新には影響しません。
> 
> ### 最終評価
> 
> 提出締切である2024年8月13日の時点で、提出物はロックされます。2024年8月13日から2024年8月27日までは、公開されていない新しい単語のセットに対してエピソードを実行し続けます。この期間中は、アクティブな3つの提出物のみがリーダーボードの対象となります。この期間の終わりに、リーダーボードが確定します。

# TrueSkill

私はレーティングシステムについてあまり知識がありません。簡単な検索によれば、$\mu$ **と** $\sigma$ パラメータを持つシステムの候補としては、[Glickoレーティングシステム](https://en.wikipedia.org/wiki/Glicko_rating_system) が考えられます。この結論に至った理由は思い出せませんが（おそらくチームプレイに関連していたのでしょうか？）、もう一つの候補は[TrueSkill](https://en.wikipedia.org/wiki/TrueSkill)です。詳細には触れませんが、興味があればWikipediaのリンクを参照してください。これは、ビデオゲームの世界で非常に人気のあるシステムです。新バージョンの[TrueSkill 2](https://www.microsoft.com/en-us/research/uploads/prod/2018/03/trueskill2.pdf)もありますが、最初の印象では、その改善点は私たちのコンペティションのような場合には重要ではなさそうです。

## インストール

幸いにも、TrueSkillを利用できる[Pythonライブラリ](https://github.com/sublee/trueskill)が[Pypi](https://pypi.org/project/trueskill/)に存在しますので、インストールしましょう。 


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install trueskill
```

</div>
<div class="column-right">

# 日本語訳

```python
!pip install trueskill
```

</div>
</details>

In [ ]:
!pip install trueskill

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Players ratings are represented by a `Rating` class, and a game is played by `rate()`.

</div>
<div class="column-right">

# 日本語訳

プレイヤーのレーティングは`Rating`クラスで表され、ゲームは`rate()`によって行われます。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from trueskill import Rating, rate
```

</div>
<div class="column-right">

# 日本語訳

```python
from trueskill import Rating, rate
```

</div>
</details>

In [ ]:
from trueskill import Rating, rate

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Let's play

Let's define four players with $\mu=600$ :

</div>
<div class="column-right">

# 日本語訳

## ゲームをプレイしよう

まず、$\mu=600$ の4人のプレイヤーを定義します：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
player1 = Rating(mu=600)
player2 = Rating(mu=600)
player3 = Rating(mu=600)
player4 = Rating(mu=600)

teamA = [ player1, player2 ]
teamB = [ player3, player4 ]
```

</div>
<div class="column-right">

# 日本語訳

```python
player1 = Rating(mu=600)
player2 = Rating(mu=600)
player3 = Rating(mu=600)
player4 = Rating(mu=600)

teamA = [ player1, player2 ]
teamB = [ player3, player4 ]
```

</div>
</details>

In [ ]:
player1 = Rating(mu=600)
player2 = Rating(mu=600)
player3 = Rating(mu=600)
player4 = Rating(mu=600)

teamA = [ player1, player2 ]
teamB = [ player3, player4 ]

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Now let them play a game :

</div>
<div class="column-right">

# 日本語訳

さて、彼らにゲームをプレイさせてみましょう：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
rate(
    [ teamA, teamB ],
    [ 0, 1 ]  # teamA is ranked lower than teamB, so teamA won
)
```

</div>
<div class="column-right">

# 日本語訳

```python
rate(
    [ teamA, teamB ],
    [ 0, 1 ]  # teamAはteamBよりもレーティングが低いため、teamAが勝利
)
```

</div>
</details>

In [ ]:
rate(
    [ teamA, teamB ],
    [ 0, 1 ]  # teamAはteamBよりもレーティングが低いため、teamAが勝利
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

The `rate()` function returns the new ratings for each players, by teams.

## Simple display function

It will be convenient to define a function to help reading the consequences of a game :

</div>
<div class="column-right">

# 日本語訳

`rate()`関数は、チームごとのプレイヤーの新しいレーティングを返します。

## 簡易表示関数

ゲームの結果を読み取りやすくするための関数を定義しておくと便利です：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from typing import Tuple

def play_game(
    teamA: Tuple[Rating, Rating],
    teamB: Tuple[Rating, Rating],
    outcome: Tuple[int, int]
):
    """
    Play a game between two teams and display the effect on the players' ratings.
    
    Params
    ------
        teamA: Tuple[Rating, Rating]
            A team of two players.
        teamB: Tuple[Rating, Rating]
            A team of two players.
        outcome: Tuple[int, int]
            The ranking of the players in the game (0: first, 1: second)
    """
    new_ratings = rate(
        [ teamA, teamB ],
        outcome
    )
    for old,new in zip(teamA, new_ratings[0]):
        delta = int(new.mu - old.mu)
        print(f"{old.mu:.0f} -> {new.mu:.0f} ({'+' if delta >=0 else ''}{delta})")
    print("vs")
    for old,new in zip(teamB, new_ratings[1]):
        delta = int(new.mu - old.mu)
        print(f"{old.mu:.0f} -> {new.mu:.0f} ({'+' if delta >=0 else ''}{delta})")
```

</div>
<div class="column-right">

# 日本語訳

```python
from typing import Tuple

def play_game(
    teamA: Tuple[Rating, Rating],
    teamB: Tuple[Rating, Rating],
    outcome: Tuple[int, int]
):
    """
    2つのチームの間でゲームをプレイし、プレイヤーのレーティングの変化を表示します。
    
    パラメータ
    --------
        teamA: Tuple[Rating, Rating]
            2人のプレイヤーからなるチーム。
        teamB: Tuple[Rating, Rating]
            2人のプレイヤーからなるチーム。
        outcome: Tuple[int, int]
            ゲーム内のプレイヤーのランキング（0: 最初, 1: 2番目）
    """
    new_ratings = rate(
        [ teamA, teamB ],
        outcome
    )
    for old,new in zip(teamA, new_ratings[0]):
        delta = int(new.mu - old.mu)
        print(f"{old.mu:.0f} -> {new.mu:.0f} ({'+' if delta >=0 else ''}{delta})")
    print("vs")
    for old,new in zip(teamB, new_ratings[1]):
        delta = int(new.mu - old.mu)
        print(f"{old.mu:.0f} -> {new.mu:.0f} ({'+' if delta >=0 else ''}{delta})")
```

</div>
</details>

In [ ]:
from typing import Tuple

def play_game(
    teamA: Tuple[Rating, Rating],
    teamB: Tuple[Rating, Rating],
    outcome: Tuple[int, int]
):
    """
    2つのチームの間でゲームをプレイし、プレイヤーのレーティングの変化を表示します。
    
    パラメータ
    --------
        teamA: Tuple[Rating, Rating]
            2人のプレイヤーからなるチーム。
        teamB: Tuple[Rating, Rating]
            2人のプレイヤーからなるチーム。
        outcome: Tuple[int, int]
            ゲーム内のプレイヤーのランキング（0: 最初, 1: 2番目）
    """
    new_ratings = rate(
        [ teamA, teamB ],
        outcome
    )
    for old,new in zip(teamA, new_ratings[0]):
        delta = int(new.mu - old.mu)
        print(f"{old.mu:.0f} -> {new.mu:.0f} ({'+' if delta >=0 else ''}{delta})")
    print("vs")
    for old,new in zip(teamB, new_ratings[1]):
        delta = int(new.mu - old.mu)
        print(f"{old.mu:.0f} -> {new.mu:.0f} ({'+' if delta >=0 else ''}{delta})")

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Applying it on our previous players :

</div>
<div class="column-right">

# 日本語訳

それを以前のプレイヤーに適用してみましょう：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
play_game(teamA, teamB, [0, 1])
```

</div>
<div class="column-right">

# 日本語訳

```python
play_game(teamA, teamB, [0, 1])
```

</div>
</details>

In [ ]:
play_game(teamA, teamB, [0, 1])

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Finding sigma

Now it will quick become obvious that the competition doesn't use the $\sigma$ default value.  The TrueSkill recommandation is to set $\sigma = \frac{\mu}{3}$, but after playing a bit with these parameters I believe the relation for this competition has rather been set to $\sigma = \frac{\mu}{2} = 300$.

Indeed, let's try to reproduce the outcome of an actual episode.  Here is the first win of my first submission :

> [1st] gguillard 600 (+123) vs [1st] JavaZero 570 (+34) vs
>
> [3rd] atom1231 577 (-72) vs [3rd] KKY 464 (-44)

With a little trial and error, and assuming my $\sigma$ would not be very far than its initialization at 300, it is quick and easy to find a matching combination of parameters :

</div>
<div class="column-right">

# 日本語訳

## シグマを見つける

ここで、コンぺティションがデフォルトの$\sigma$値を使用していないことは明らかとなります。TrueSkillの推奨値は$\sigma = \frac{\mu}{3}$ですが、これらのパラメータを少し試した結果、コンペティションの$\sigma$はむしろ$\sigma = \frac{\mu}{2} = 300$に設定されているようです。

実際に、実際のエピソードの結果を再現しようとすると、私の最初の提出の最初の勝利は次のようになります：

> [1位] gguillard 600 (+123) vs [1位] JavaZero 570 (+34) vs
>
> [3位] atom1231 577 (-72) vs [3位] KKY 464 (-44)

少し試行錯誤しながら、私の$\sigma$が300の初期値から大きく外れていないだろうと仮定すると、一致するパラメータの組み合わせを見つけるのはすぐに簡単でした：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
play_game(
    [
        Rating(mu=600, sigma=295),
        Rating(mu=570, sigma=156)
    ],
    [
        Rating(mu=577, sigma=225),
        Rating(mu=464, sigma=176)
    ],
    [ 0, 1 ]  # first team won
)
```

</div>
<div class="column-right">

# 日本語訳

```python
play_game(
    [
        Rating(mu=600, sigma=295),
        Rating(mu=570, sigma=156)
    ],
    [
        Rating(mu=577, sigma=225),
        Rating(mu=464, sigma=176)
    ],
    [ 0, 1 ]  # 1番目のチームが勝利
)
```

</div>
</details>

In [ ]:
play_game(
    [
        Rating(mu=600, sigma=295),
        Rating(mu=570, sigma=156)
    ],
    [
        Rating(mu=577, sigma=225),
        Rating(mu=464, sigma=176)
    ],
    [ 0, 1 ]  # 1番目のチームが勝利
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## The draw probability

So is the scoring system TrueSkill with $\mu=600$ and $\sigma=300$ ?  Well, not quite.  These settings don't work well in case of draws, the variations are much larger than what can be observed in the competition (typically 0 to ± a few units) :

</div>
<div class="column-right">

# 日本語訳

## 引き分けの確率

スコアリングシステムはTrueSkillの$\mu=600$と$\sigma=300$ですか？ さて、実際にはそうではありません。これらの設定は引き分けの場合にはあまりうまく機能せず、変動はコンペティションで観察されるよりも大きく（典型的には0から±数ユニット）なります：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
play_game(
    [
        Rating(mu=600, sigma=295),
        Rating(mu=570, sigma=156)
    ],
    [
        Rating(mu=577, sigma=225),
        Rating(mu=464, sigma=176)
    ],
    [ 1, 1 ]  # no winner
)
```

</div>
<div class="column-right">

# 日本語訳

```python
play_game(
    [
        Rating(mu=600, sigma=295),
        Rating(mu=570, sigma=156)
    ],
    [
        Rating(mu=577, sigma=225),
        Rating(mu=464, sigma=176)
    ],
    [ 1, 1 ]  # 勝者なし
)
```

</div>
</details>

In [ ]:
play_game(
    [
        Rating(mu=600, sigma=295),
        Rating(mu=570, sigma=156)
    ],
    [
        Rating(mu=577, sigma=225),
        Rating(mu=464, sigma=176)
    ],
    [ 1, 1 ]  # 勝者なし
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Reading again the rules, the keypoint is here :

> We also reduce the σ terms relative to the amount of information gained by the result.

At first I played a bit with parameters and thought $\sigma$ was manually reduced by a factor 100 for draws, it seemed to work fairly well.  Then reading `help(trueskill)` I figured it may already be taken into account within TrueSkill, since the V and W functions, responsible for the $\mu$ and $\sigma$ update, respectively, have a "win" **and** a "draw" version.

Actually, one can define a TrueSkill environment with a few more parameters, including `draw_probability`.  That's interesting, because we all know the draw probability for a game in this competition is… *fairly high*.

Let's compute it, we'll need it for our simulation.  We'll use @waechter's [LLM 20 Questions - Games dataset](https://www.kaggle.com/code/waechter/llm-20-questions-games-dataset).

</div>
<div class="column-right">

# 日本語訳

ルールを再度読み返すと、ここが重要なポイントです：

> 私たちは、結果によって得られた情報量に応じて、σの項も減少させます。

最初はパラメータをいじっていて、引き分けの場合にσが100の係数で手動で減少されると思っていましたが、これはかなりうまく機能するようです。その後、`help(trueskill)`を読むと、これはすでにTrueSkill内で考慮されている可能性があり、μとσの更新を担当するVとW関数には「勝利」**および**「引き分け」バージョンがあります。

実際には、`draw_probability`を含むいくつかの追加パラメータを使ってTrueSkill環境を定義できます。これは興味深い点であり、このコンペティションでのゲームの引き分けの確率は… *かなり高い* ことを私たち全員が知っています。

これを計算してみましょう、そのためには@waechterの[LLM 20 Questions - Games dataset](https://www.kaggle.com/code/waechter/llm-20-questions-games-dataset)を使用します。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import pandas as pd
from ast import literal_eval

games_df = pd.read_csv(
    "/kaggle/input/llm-20-questions-games-dataset/LLM-20Questions-games.csv",
    converters = {
        col_list: literal_eval
        for col_list in ["answers", "questions", "guesses"]
    }
)
games_df = games_df[games_df["guesser_SubmissionId"]!=0]  # it seems that submissions of the day are not yet labelled
games_df["CreateTime"] = pd.to_datetime(games_df["CreateTime"], format="%m/%d/%Y %H:%M:%S")
```

</div>
<div class="column-right">

# 日本語訳

```python
import pandas as pd
from ast import literal_eval

games_df = pd.read_csv(
    "/kaggle/input/llm-20-questions-games-dataset/LLM-20Questions-games.csv",
    converters = {
        col_list: literal_eval
        for col_list in ["answers", "questions", "guesses"]
    }
)
games_df = games_df[games_df["guesser_SubmissionId"]!=0]  # その日の提出物がまだラベル付けされていないようです
games_df["CreateTime"] = pd.to_datetime(games_df["CreateTime"], format="%m/%d/%Y %H:%M:%S")
```

</div>
</details>

In [ ]:
import pandas as pd
from ast import literal_eval

games_df = pd.read_csv(
    "/kaggle/input/llm-20-questions-games-dataset/LLM-20Questions-games.csv",
    converters = {
        col_list: literal_eval
        for col_list in ["answers", "questions", "guesses"]
    }
)
games_df = games_df[games_df["guesser_SubmissionId"]!=0]  # その日の提出物がまだラベル付けされていないようです
games_df["CreateTime"] = pd.to_datetime(games_df["CreateTime"], format="%m/%d/%Y %H:%M:%S")

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

This dataframe contains two rows per game, one for each team, and a boolean "guessed" column.  In case of a draw, "guessed" is the same for both teams (although we could safely ignore the 3 games where both teams won at the same round…).

</div>
<div class="column-right">

# 日本語訳

このデータフレームには各ゲームについて2行が含まれています。一つはそれぞれのチームの行であり、`guessed`というブール値の列を持っています。引き分けの場合、 `guessed` は両チーム同じです（ただし、両チームが同じラウンドで勝った3ゲームは無視しても問題ありません）。 


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
games_df.groupby("game_num")["guessed"].sum().value_counts()
```

</div>
<div class="column-right">

# 日本語訳

```python
games_df.groupby("game_num")["guessed"].sum().value_counts()
```

</div>
</details>

In [ ]:
games_df.groupby("game_num")["guessed"].sum().value_counts()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
(144607 + 3) / (144607 + 1208 + 3)
```

</div>
<div class="column-right">

# 日本語訳

```python
(144607 + 3) / (144607 + 1208 + 3)
```

</div>
</details>

In [ ]:
(144607 + 3) / (144607 + 1208 + 3)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Therefore, the probability of a draw is : $p \simeq 0.9917$.  Fairly high indeed…  We'll update it later with more accurate data (spoiler : it'll only get worse).

## Beta and tau

The documentation also mentions two other parameters that may be of interest :

>  :param beta: the distance which guarantees about 76% chance of winning.
>               The recommended value is a half of ``sigma``.
>
>  :param tau: the dynamic factor which restrains a fixation of rating.  The
>              recommended value is ``sigma`` per cent.

Let's use the recommended values, namely $\beta = \frac{\sigma}{2} = 150$ and $\tau = \frac{\sigma}{100} = 3$.

Now let's define our new TrueSkill environment :

</div>
<div class="column-right">

# 日本語訳

したがって、引き分けの確率は $p \simeq 0.9917$ です。確かにかなり高いです…。後で、より正確なデータで更新する必要があります（ネタバレ：悪化するだけです）。

## ベータとタウ

ドキュメンテーションには、興味深い他の2つのパラメーターも言及されています：

>  :param beta: 勝利の約76％の確率を保証する距離。
>               推奨値は`sigma`の半分です。
>
>  :param tau: レーティングの固定化を制限するダイナミックファクター。
>              推奨値は`sigma`パーセントです。

推奨値である$\beta = \frac{\sigma}{2} = 150$および$\tau = \frac{\sigma}{100} = 3$を使用しましょう。

さて、新しいTrueSkill環境を定義しましょう：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from trueskill import setup

setup(
    mu = 600,
    sigma = 300,
    beta = 150,
    tau = 3,
    draw_probability = 0.9917
)
```

</div>
<div class="column-right">

# 日本語訳

```python
from trueskill import setup

setup(
    mu = 600,
    sigma = 300,
    beta = 150,
    tau = 3,
    draw_probability = 0.9917
)
```

</div>
</details>

In [ ]:
from trueskill import setup

setup(
    mu = 600,
    sigma = 300,
    beta = 150,
    tau = 3,
    draw_probability = 0.9917
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

It takes a little bit of adjustment again to get the right $\sigma$ :

</div>
<div class="column-right">

# 日本語訳

再度、適切な$\sigma$を見つけるための調整が必要です：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
play_game(
    [
        Rating(mu=600, sigma=144),
        Rating(mu=570, sigma=76)
    ],
    [
        Rating(mu=577, sigma=109),
        Rating(mu=464, sigma=85)
    ],
    [ 0, 1 ]  # first team won
)
```

</div>
<div class="column-right">

# 日本語訳

```python
play_game(
    [
        Rating(mu=600, sigma=144),
        Rating(mu=570, sigma=76)
    ],
    [
        Rating(mu=577, sigma=109),
        Rating(mu=464, sigma=85)
    ],
    [ 0, 1 ]  # 最初のチームが勝利
)
```

</div>
</details>

In [ ]:
play_game(
    [
        Rating(mu=600, sigma=144),
        Rating(mu=570, sigma=76)
    ],
    [
        Rating(mu=577, sigma=109),
        Rating(mu=464, sigma=85)
    ],
    [ 0, 1 ]  # 最初のチームが勝利
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

But then, ratings update after a draw are much more consistent with what we actually observe in the competition :

</div>
<div class="column-right">

# 日本語訳

しかし、引き分けの後のレーティングの更新は、実際にコンペティションで観察されるものと一致します：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
play_game(
    [
        Rating(mu=600, sigma=144),
        Rating(mu=570, sigma=76)
    ],
    [
        Rating(mu=577, sigma=109),
        Rating(mu=464, sigma=85)
    ],
    [ 1, 1 ]  # no winner
)
```

</div>
<div class="column-right">

# 日本語訳

```python
play_game(
    [
        Rating(mu=600, sigma=144),
        Rating(mu=570, sigma=76)
    ],
    [
        Rating(mu=577, sigma=109),
        Rating(mu=464, sigma=85)
    ],
    [ 1, 1 ]  # 勝者なし
)
```

</div>
</details>

In [ ]:
play_game(
    [
        Rating(mu=600, sigma=144),
        Rating(mu=570, sigma=76)
    ],
    [
        Rating(mu=577, sigma=109),
        Rating(mu=464, sigma=85)
    ],
    [ 1, 1 ]  # 勝者なし
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

It looks to me that we managed to construct a scoring system quite similar to the one used by the LLM 20 Questions competition.  It is possible that the parameters are not *exactly* those ones, and anyway the draw probability couldn't have been deduced before a certain number of games (maybe it's updated continuously ? – it also accepts a function, BTW), but I would be surprised if the ability of these settings to reproduce the results of a game was a coincidence.

# Modeling the competition

## Winning probabilities

Now what about generating a full board of players, teaming them up randomly and having them play ? :D

Let's first extract some realistic probabilities of winning.  Using the *LLM 20 Questions - Games dataset* again, we'll compute the `nb_guessed / nb_played` ratio for each submission.

But in order to get a realistic distribution of skills, we will first filter out submissions with a small number of games played (say 10) :

</div>
<div class="column-right">

# 日本語訳

これで、私たちはLLM 20 Questionsコンペティションで使用されているスコアリングシステムに非常に似たものを構築できたようです。おそらく、パラメーターは正確にその値ではないかもしれませんし、引き分けの確率も数回のゲーム後にしか導き出せません（おそらく継続的に更新されているのかもしれませんが、関数も受け付けますので）、しかし、これらの設定がゲームの結果を再現する能力は偶然とは思えません。

# コンペティションのモデル化

## 勝利の確率

それでは、プレイヤーの全テーブルをランダムに組んでゲームをプレイしてみるのはどうでしょうか？ :D

初めに、*LLM 20 Questions - Games dataset*を使用して、各提出物について「nb_guessed / nb_played」比率を計算して実際的な勝利確率を取得しましょう。

ただし、まずは少ないゲーム数をプレイしている提出物（例えば10以下）をフィルタリングする必要があります：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
nb_games_played = games_df.groupby("guesser_SubmissionId").size()
nb_games_played = nb_games_played[nb_games_played>=10]
```

</div>
<div class="column-right">

# 日本語訳

```python
nb_games_played = games_df.groupby("guesser_SubmissionId").size()
nb_games_played = nb_games_played[nb_games_played>=10]
```

</div>
</details>

In [ ]:
nb_games_played = games_df.groupby("guesser_SubmissionId").size()
nb_games_played = nb_games_played[nb_games_played>=10]

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
games_df = games_df[games_df["guesser_SubmissionId"].isin(nb_games_played.index)]
```

</div>
<div class="column-right">

# 日本語訳

```python
games_df = games_df[games_df["guesser_SubmissionId"].isin(nb_games_played.index)]
```

</div>
</details>

In [ ]:
games_df = games_df[games_df["guesser_SubmissionId"].isin(nb_games_played.index)]

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Since the competition only allows three submissions per competitor, we also have to remove all bots but the last three — some people have more than 100 submissions !

</div>
<div class="column-right">

# 日本語訳

コンペティションでは1人あたり最大3の提出しか許可されないため、最後の3つを残して他のボットをすべて削除する必要があります。　一部の人は100以上の提出物を持っている場合があります！


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
games_df.groupby("guesser")["guesser_SubmissionId"].nunique().sort_values(ascending=False).head()
```

</div>
<div class="column-right">

# 日本語訳

```python
games_df.groupby("guesser")["guesser_SubmissionId"].nunique().sort_values(ascending=False).head()
```

</div>
</details>

In [ ]:
games_df.groupby("guesser")["guesser_SubmissionId"].nunique().sort_values(ascending=False).head()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
last_bots = games_df.sort_values(
    by = "CreateTime"  # don't sort by "guesser" here because some submissions have several names
).drop_duplicates(
    subset = "guesser_SubmissionId",
    keep = "last"
).groupby(
    [ "guesser" ]
)[[
    "guesser",
    "guesser_SubmissionId",
    "CreateTime"
]].tail(3).sort_values(
    by = "guesser"
)["guesser_SubmissionId"].values
```

</div>
<div class="column-right">

# 日本語訳

```python
last_bots = games_df.sort_values(
    by = "CreateTime"  # 「guesser」でソートすると、一部の提出物に複数の名前が付いているためここではソートしません
).drop_duplicates(
    subset = "guesser_SubmissionId",
    keep = "last"
).groupby(
    [ "guesser" ]
)[[
    "guesser",
    "guesser_SubmissionId",
    "CreateTime"
]].tail(3).sort_values(
    by = "guesser"
)["guesser_SubmissionId"].values
```

</div>
</details>

In [ ]:
last_bots = games_df.sort_values(
    by = "CreateTime"  # 「guesser」でソートすると、一部の提出物に複数の名前が付いているためここではソートしません
).drop_duplicates(
    subset = "guesser_SubmissionId",
    keep = "last"
).groupby(
    [ "guesser" ]
)[[
    "guesser",
    "guesser_SubmissionId",
    "CreateTime"
]].tail(3).sort_values(
    by = "guesser"
)["guesser_SubmissionId"].values

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
last_games_df = games_df[games_df["guesser_SubmissionId"].isin(last_bots)]
```

</div>
<div class="column-right">

# 日本語訳

```python
last_games_df = games_df[games_df["guesser_SubmissionId"].isin(last_bots)]
```

</div>
</details>

In [ ]:
last_games_df = games_df[games_df["guesser_SubmissionId"].isin(last_bots)]

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Time to update our draw probability :

</div>
<div class="column-right">

# 日本語訳

引き分けの確率を更新する時が来ました：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
last_games_df.groupby("game_num")["guessed"].sum().value_counts()
```

</div>
<div class="column-right">

# 日本語訳

```python
last_games_df.groupby("game_num")["guessed"].sum().value_counts()
```

</div>
</details>

In [ ]:
last_games_df.groupby("game_num")["guessed"].sum().value_counts()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
(123874 + 1) / (123874 + 546 + 1)
```

</div>
<div class="column-right">

# 日本語訳

```python
(123874 + 1) / (123874 + 546 + 1)
```

</div>
</details>

In [ ]:
(123874 + 1) / (123874 + 546 + 1)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

As mentioned earlier, it's even worse than before, with the probability of a draw being $p \simeq 0.9956$…

We'll need to distinguish between "good bots" and "dummy bots" to compute the actual skills distribution.

</div>
<div class="column-right">

# 日本語訳

先ほど述べたように、引き分けの確率は $p \simeq 0.9956$ とさらに悪化しました…

実際のスキル分布を計算するためには、「良いボット」と「ダミーボット」を区別する必要があります。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
goodbots = set(
    last_games_df[
        last_games_df.groupby("guesser_SubmissionId")["guessed"].transform("any")
    ]["guesser_SubmissionId"].unique()
)
```

</div>
<div class="column-right">

# 日本語訳

```python
goodbots = set(
    last_games_df[
        last_games_df.groupby("guesser_SubmissionId")["guessed"].transform("any")
    ]["guesser_SubmissionId"].unique()
)
```

</div>
</details>

In [ ]:
goodbots = set(
    last_games_df[
        last_games_df.groupby("guesser_SubmissionId")["guessed"].transform("any")
    ]["guesser_SubmissionId"].unique()
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
badbots = set(last_games_df["guesser_SubmissionId"].unique()) - goodbots
```

</div>
<div class="column-right">

# 日本語訳

```python
badbots = set(last_games_df["guesser_SubmissionId"].unique()) - goodbots
```

</div>
</details>

In [ ]:
badbots = set(last_games_df["guesser_SubmissionId"].unique()) - goodbots

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
len(goodbots), len(badbots)
```

</div>
<div class="column-right">

# 日本語訳

```python
len(goodbots), len(badbots)
```

</div>
</details>

In [ ]:
len(goodbots), len(badbots)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Now let's consider every team of good bots.  We don't care about the opponents being dumb, because it only takes a reliable teammate to find the keyword.  We will also discard sessions in which the team didn't have time to find the keyword because the other team did before, as this would only bias our simulation.  On the other hand, we keep instances in which the keyword was guessed although the teammate is in the badbots category, because it may just be that the teammate didn't have a chance to win yet.  Incidentally, removing these cases would drastically reduce the statistics.

</div>
<div class="column-right">

# 日本語訳

次に、すべての良いボットのチームを考えてみましょう。相手がダミーの場合は問題ありません。なぜなら、信頼できるチームメイトが単語を見つけてくれるからです。また、相手が見つける前にチームが単語を見つける時間がなかったため、そのセッションを無視することはありません。逆に、チームメイトが勝つ機会がなかった可能性があるため、良いボットと悪いボットのチームでも、キーワードを当てた場合は記録に残します。ちなみに、これらを削除すると、統計が大幅に減少します。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
fair_games = last_games_df["guesser_SubmissionId"].isin(goodbots) * (
    last_games_df["answerer_SubmissionId"].isin(goodbots) + last_games_df["guessed"]
)
fair_games_df = last_games_df.loc[fair_games]
fair_games_df = fair_games_df[(fair_games_df["guessed"]) | (fair_games_df["nb_round"]==20)]
```

</div>
<div class="column-right">

# 日本語訳

```python
fair_games = last_games_df["guesser_SubmissionId"].isin(goodbots) * (
    last_games_df["answerer_SubmissionId"].isin(goodbots) + last_games_df["guessed"]
)
fair_games_df = last_games_df.loc[fair_games]
fair_games_df = fair_games_df[(fair_games_df["guessed"]) | (fair_games_df["nb_round"]==20)]
```

</div>
</details>

In [ ]:
fair_games = last_games_df["guesser_SubmissionId"].isin(goodbots) * (
    last_games_df["answerer_SubmissionId"].isin(goodbots) + last_games_df["guessed"]
)
fair_games_df = last_games_df.loc[fair_games]
fair_games_df = fair_games_df[(fair_games_df["guessed"]) | (fair_games_df["nb_round"]==20)]

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
skills_df = fair_games_df.groupby("guesser_SubmissionId").agg(
    guessed = ("guessed", "sum"),
    played = ("guessed", "size")
).sort_index()
skills_df["skill"] = skills_df["guessed"] / skills_df["played"]
skills_df.sort_values(by = "skill", ascending = False, inplace = True)
skills_df.head()
```

</div>
<div class="column-right">

# 日本語訳

```python
skills_df = fair_games_df.groupby("guesser_SubmissionId").agg(
    guessed = ("guessed", "sum"),
    played = ("guessed", "size")
).sort_index()
skills_df["skill"] = skills_df["guessed"] / skills_df["played"]
skills_df.sort_values(by = "skill", ascending = False, inplace = True)
skills_df.head()
```

</div>
</details>

In [ ]:
skills_df = fair_games_df.groupby("guesser_SubmissionId").agg(
    guessed = ("guessed", "sum"),
    played = ("guessed", "size")
).sort_index()
skills_df["skill"] = skills_df["guessed"] / skills_df["played"]
skills_df.sort_values(by = "skill", ascending = False, inplace = True)
skills_df.head()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Let's remove bots with less than $N = 10$ fair games in order to avoid unrealistic probabilities (we already did it for *all* games, but *fair* games is an other story) :

</div>
<div class="column-right">

# 日本語訳

不正確な確率を避けるために、$N = 10$ の公正ゲームを持たないボットを削除しましょう。すでに全ゲームに対して行いましたが、公正ゲームに対しても行う必要があります。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
skills_df = skills_df[skills_df["played"]>=10]
skills_df.head()
```

</div>
<div class="column-right">

# 日本語訳

```python
skills_df = skills_df[skills_df["played"]>=10]
skills_df.head()
```

</div>
</details>

In [ ]:
skills_df = skills_df[skills_df["played"]>=10]
skills_df.head()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Finally, let's increase the dynamic range of skills by allowing the best guesser to be a superguesser with 98 % of chances to find the keyword.  Thats very optimistic !  But this will also be representative of the evolution towards better bots as we approach the end of the competition.

</div>
<div class="column-right">

# 日本語訳

最後に、キーワードを見つける確率が98%のスーパーチューザーを設定することによって、スキルのダイナミックレンジを拡大しましょう。これは非常に楽観的ですが、コンペティションの終わりに向けて、より優れたボットが進化する様子を示すとも言えます。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
BEST_SKILL = .98
skills_df["skill"] *= BEST_SKILL / skills_df["skill"].iloc[0]
```

</div>
<div class="column-right">

# 日本語訳

```python
BEST_SKILL = .98
skills_df["skill"] *= BEST_SKILL / skills_df["skill"].iloc[0]
```

</div>
</details>

In [ ]:
BEST_SKILL = .98
skills_df["skill"] *= BEST_SKILL / skills_df["skill"].iloc[0]

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Now we have a somehow realistic representation of the global skill distribution of the current (legit) bots.  Let's have a look at it :

</div>
<div class="column-right">

# 日本語訳

これで、現在の（正当な）ボットのグローバルなスキル分布のある程度現実的な表現が得られました。これを見てみましょう：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import matplotlib.pyplot as plt
```

</div>
<div class="column-right">

# 日本語訳

```python
import matplotlib.pyplot as plt
```

</div>
</details>

In [ ]:
import matplotlib.pyplot as plt

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
plt.figure(figsize=(12,6))
_ = plt.hist(skills_df["skill"], bins=15)
_ = plt.xlabel("Probability to find the keyword against a fair teammate")
_ = plt.ylabel("Number of bots")
_ = plt.title("Good bots skill distribution")
```

</div>
<div class="column-right">

# 日本語訳

```python
plt.figure(figsize=(12,6))
_ = plt.hist(skills_df["skill"], bins=15)
_ = plt.xlabel("公正なチームメイトに対するキーワード発見の確率")
_ = plt.ylabel("ボットの数")
_ = plt.title("良いボットのスキル分布")
```

</div>
</details>

In [ ]:
plt.figure(figsize=(12,6))
_ = plt.hist(skills_df["skill"], bins=15)
_ = plt.xlabel("公正なチームメイトに対するキーワード発見の確率")
_ = plt.ylabel("ボットの数")
_ = plt.title("良いボットのスキル分布")

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

That should lead to a clear leaderboard !

## A Player class

Now let's create a Player class in order to keep track of the ratings history :

</div>
<div class="column-right">

# 日本語訳

これにより、明確なリーダーボードが得られるはずです！

## プレイヤークラス

次に、レーティング履歴を追跡できるPlayerクラスを作成しましょう：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from typing import Optional
from trueskill import TrueSkill

class Player:
    def __init__(
        self,
        skill: float,
        env: Optional[TrueSkill] = None  # ignore this for now, this will be useful later
    ):
        self.skill = skill
        self.rating = Rating() if env is None else env.Rating()  # start with environment default values, i.e. (600,300)
        self.history = []
        
    def update(self, new_rating):
        self.history.append(self.rating)
        self.rating = new_rating
```

</div>
<div class="column-right">

# 日本語訳

```python
from typing import Optional
from trueskill import TrueSkill

class Player:
    def __init__(
        self,
        skill: float,
        env: Optional[TrueSkill] = None  # 今は無視しますが、後で使えるようにするためです
    ):
        self.skill = skill
        self.rating = Rating() if env is None else env.Rating()  # 環境のデフォルト値（つまり600,300）で開始
        self.history = []
        
    def update(self, new_rating):
        self.history.append(self.rating)
        self.rating = new_rating
```

</div>
</details>

In [ ]:
from typing import Optional
from trueskill import TrueSkill

class Player:
    def __init__(
        self,
        skill: float,
        env: Optional[TrueSkill] = None  # 今は無視しますが、後で使えるようにするためです
    ):
        self.skill = skill
        self.rating = Rating() if env is None else env.Rating()  # 環境のデフォルト値（つまり600,300）で開始
        self.history = []
        
    def update(self, new_rating):
        self.history.append(self.rating)
        self.rating = new_rating

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## A pool of players

We can then build a pool of players, some with a range of realistic skills, along with a number of dummy bots.  Since there was a significant reduction of the number of good bots considered for the skills evaluation, it seems fair to apply the same reduction ratio to the number of bad bots as well, but anyway it shouldn't change much the outcome of the simulation (actually it may get worse…).  Play with it if you wish.

</div>
<div class="column-right">

# 日本語訳

## プレイヤーのプール

次に、現実的なスキルの範囲を持つプレイヤーのプールを構築し、ダミーボットの数を追加します。スキル評価のために考慮される良いボットの数が大幅に削減されたため、数の減少比を悪いボットにも適用するのが妥当でしょう。しかし、これはシミュレーションの結果にそれほど影響を与えないはずです（実際には悪化させる可能性があります）。遊んでみてください。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
reduction_ratio = len(skills_df) / len(goodbots)

players = { ii: Player(s) for ii,s in enumerate(skills_df["skill"].values) }
players.update({ len(players)+ii: Player(0) for ii in range(int(len(badbots)*reduction_ratio)) })

len(players)
```

</div>
<div class="column-right">

# 日本語訳

```python
reduction_ratio = len(skills_df) / len(goodbots)

players = { ii: Player(s) for ii,s in enumerate(skills_df["skill"].values) }
players.update({ len(players)+ii: Player(0) for ii in range(int(len(badbots)*reduction_ratio)) })

len(players)
```

</div>
</details>

In [ ]:
reduction_ratio = len(skills_df) / len(goodbots)

players = { ii: Player(s) for ii,s in enumerate(skills_df["skill"].values) }
players.update({ len(players)+ii: Player(0) for ii in range(int(len(badbots)*reduction_ratio)) })

len(players)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

We still have more players than in the actual leaderboard, but that's the number of bots, remember each competitor on the leaderboard may have up to three active submissions.

## A game function

Finally, we need to define a game function taking into account the winning probability.

</div>
<div class="column-right">

# 日本語訳

実際のリーダーボードにいるプレイヤーよりもまだ多くのプレイヤーがいますが、これはボットの数です。リーダーボードの競技者は最大3つのアクティブな提出物を持つことができます。

## ゲーム関数

最後に、勝利の確率を考慮したゲーム関数を定義する必要があります。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import random
```

</div>
<div class="column-right">

# 日本語訳

```python
import random
```

</div>
</details>

In [ ]:
import random

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def play_match(
    teamA: Tuple[Player, Player],
    teamB: Tuple[Player, Player],
    env: Optional[TrueSkill] = None,
    debug: bool = False
):
    """
    Plays a match between two teams.
    
    Params
    ------
        teamA: Tuple[Player, Player]
            A team of two players.
        teamB: Tuple[Player, Player]
            Another team of two players.
        debug: boolean
            A flag for debugging.
    """
    if debug:
        print("Players skills :", [p.skill for p in teamA+teamB])
        print("Players ratings :", [tuple(p.rating for p in teamA), tuple(p.rating for p in teamB)])
    # for each team, if any player is a dummy bot, the team fails to find the keyword
    # else the team finds the keyword if its guesser's skill is higher than a random number
    # (beware of the reversed logic in the code)
    ranks = [
        1 if teamA[0].skill * teamA[1].skill == 0 or random.random() > teamA[0].skill else 0,
        1 if teamB[0].skill * teamB[1].skill == 0 or random.random() > teamB[0].skill else 0
    ]
    
    # new ratings are the results of the game
    rating_function = rate if env is None else env.rate
    new_ratings = rating_function(
        [
            [ p.rating for p in teamA ],
            [ p.rating for p in teamB ]
        ],
        ranks
    )
    
    # update all players' ratings
    for p,r in zip(teamA, new_ratings[0]):
        p.update(r)
    for p,r in zip(teamB, new_ratings[1]):
        p.update(r)
        
    if debug:
        print("Game outcome :", ranks)
        print("New ratings :", new_ratings)
    
    return
```

</div>
<div class="column-right">

# 日本語訳

```python
def play_match(
    teamA: Tuple[Player, Player],
    teamB: Tuple[Player, Player],
    env: Optional[TrueSkill] = None,
    debug: bool = False
):
    """
    2チーム間の試合を行います。
    
    パラメータ
    ------
        teamA: Tuple[Player, Player]
            2人のプレイヤーからなるチーム。
        teamB: Tuple[Player, Player]
            もう一つの2人のプレイヤーからなるチーム。
        debug: boolean
            デバッグ用フラグ。
    """
    if debug:
        print("プレイヤーのスキル　:", [p.skill for p in teamA+teamB])
        print("プレイヤーのレーティング　:", [tuple(p.rating for p in teamA), tuple(p.rating for p in teamB)])
    # 各チームについて、どのプレイヤーがダミーボットかを判定し、ボットがキーワードを見つけられなければ失敗
    # それ以外の場合、チームの目標スキルがランダムな数よりも高ければキーワードを見つけられる
    # （影に逆の論理があることに注意）
    ranks = [
        1 if teamA[0].skill * teamA[1].skill == 0 or random.random() > teamA[0].skill else 0,
        1 if teamB[0].skill * teamB[1].skill == 0 or random.random() > teamB[0].skill else 0
    ]
    
    # 新しいレーティングはゲームの結果
    rating_function = rate if env is None else env.rate
    new_ratings = rating_function(
        [
            [ p.rating for p in teamA ],
            [ p.rating for p in teamB ]
        ],
        ranks
    )
    
    # すべてのプレイヤーのレーティングを更新
    for p,r in zip(teamA, new_ratings[0]):
        p.update(r)
    for p,r in zip(teamB, new_ratings[1]):
        p.update(r)
        
    if debug:
        print("ゲームの結果　:", ranks)
        print("新しいレーティング　:", new_ratings)
    
    return
```

</div>
</details>

In [ ]:
def play_match(
    teamA: Tuple[Player, Player],
    teamB: Tuple[Player, Player],
    env: Optional[TrueSkill] = None,
    debug: bool = False
):
    """
    2チーム間の試合を行います。
    
    パラメータ
    ------
        teamA: Tuple[Player, Player]
            2人のプレイヤーからなるチーム。
        teamB: Tuple[Player, Player]
            もう一つの2人のプレイヤーからなるチーム。
        debug: boolean
            デバッグ用フラグ。
    """
    if debug:
        print("プレイヤーのスキル　:", [p.skill for p in teamA+teamB])
        print("プレイヤーのレーティング　:", [tuple(p.rating for p in teamA), tuple(p.rating for p in teamB)])
    # 各チームについて、どのプレイヤーがダミーボットかを判定し、ボットがキーワードを見つけられなければ失敗
    # それ以外の場合、チームの目標スキルがランダムな数よりも高ければキーワードを見つけられる
    # （影に逆の論理があることに注意）
    ranks = [
        1 if teamA[0].skill * teamA[1].skill == 0 or random.random() > teamA[0].skill else 0,
        1 if teamB[0].skill * teamB[1].skill == 0 or random.random() > teamB[0].skill else 0
    ]
    
    # 新しいレーティングはゲームの結果
    rating_function = rate if env is None else env.rate
    new_ratings = rating_function(
        [
            [ p.rating for p in teamA ],
            [ p.rating for p in teamB ]
        ],
        ranks
    )
    
    # すべてのプレイヤーのレーティングを更新
    for p,r in zip(teamA, new_ratings[0]):
        p.update(r)
    for p,r in zip(teamB, new_ratings[1]):
        p.update(r)
        
    if debug:
        print("ゲームの結果　:", ranks)
        print("新しいレーティング　:", new_ratings)
    
    return

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

And voilà !  We have everything we need to start simulating the competition.  Here is a first game :

</div>
<div class="column-right">

# 日本語訳

これで、シミュレーションを開始するのに必要なすべてが揃いました。最初のゲームはこちらです：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
play_match(
    [players[0], players[1]],
    [players[2], players[3]],
    debug = True
)
```

</div>
<div class="column-right">

# 日本語訳

```python
play_match(
    [players[0], players[1]],
    [players[2], players[3]],
    debug = True
)
```

</div>
</details>

In [ ]:
play_match(
    [players[0], players[1]],
    [players[2], players[3]],
    debug = True
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Make your own competition simulation

For convenience and the ability to test different settings easily, we will define a competition class where we can parametrize the number and skills of players, the number of games and the TrueSkill settings, and which provides a dataframe with the ratings history.

</div>
<div class="column-right">

# 日本語訳

# 独自のコンペティションシミュレーションを行おう

便利さと異なる設定を簡単にテストできるように、プレイヤーの数やスキル、ゲーム数、TrueSkillの設定にパラメータを設定できるコンペティションクラスを定義します。レーティングの履歴を持つデータフレームを提供します。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from typing import List
from tqdm import tqdm
```

</div>
<div class="column-right">

# 日本語訳

```python
from typing import List
from tqdm import tqdm
```

</div>
</details>

In [ ]:
from typing import List
from tqdm import tqdm

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
class Competition:
    """
    A configurable TrueSkill competition environment.
    """
    
    def __init__(
        self,
        skill_dist: List[float],
        nb_dummy: int,
        mu: float = 600,
        sigma: float = 300,
        beta: float = 150,
        tau: float = 3,
        draw_probability: float = 0.9956,
        seed: float = 42
    ):
        """
        Constructor.
        
        Params
        ------
            skill_dist: List[float]
                The probability to find the keyword, for each "good" submission.
            nb_dummy: int
                The number of dummy bots
            mu: float
                The TrueSkill mu parameter.
            sigma: float
                The TrueSkill sigma parameter.
            beta: float
                The TrueSkill beta parameter.
            tau: float
                The TrueSkill tau parameter.
            draw_probability: float
                The TrueSkill draw_probability parameter.
            seed: float
                A random seed.
        """
        # our competition environment
        self._trueskill = TrueSkill(
            mu = mu,
            sigma = sigma,
            beta = beta,
            tau = tau,
            draw_probability = draw_probability
        )
        
        # our pool of players — and that's why we needed an "env" parameters for the Player constructor
        self.players = { ii: Player(s, self._trueskill) for ii,s in enumerate(skill_dist) }
        self.players.update({ len(self.players)+ii: Player(0, self._trueskill) for ii in range(nb_dummy) })
        
        # the random seed
        self._seed = seed
        random.seed(seed)
        
        
    def play_games(self, nb_games: int = 10000):
        """
        Simulate games and update the players history.
        
        Params
        ------
            nb_games: int
                The number of games to play.
        """
        
        for i in tqdm(range(nb_games)):
            player_ids = random.sample(range(len(self.players)), 4)  # let's pick 4 players randomly
            try:
                play_match(
                    [
                        self.players[player_ids[0]],
                        self.players[player_ids[1]]
                    ],
                    [
                        self.players[player_ids[2]],
                        self.players[player_ids[3]]
                    ],
                    env = self._trueskill
                )
            except:
                continue
```

</div>
<div class="column-right">

# 日本語訳

```python
class Competition:
    """
    TrueSkillのコンペティション環境を設定できます。
    """
    
    def __init__(
        self,
        skill_dist: List[float],
        nb_dummy: int,
        mu: float = 600,
        sigma: float = 300,
        beta: float = 150,
        tau: float = 3,
        draw_probability: float = 0.9956,
        seed: float = 42
    ):
        """
        コンストラクター。
        
        パラメータ
        ------
            skill_dist: List[float]
                各「良い」提出のキーワード発見の確率。
            nb_dummy: int
                ダミーボットの数。
            mu: float
                TrueSkillのmuパラメータ。
            sigma: float
                TrueSkillのsigmaパラメータ。
            beta: float
                TrueSkillのbetaパラメータ。
            tau: float
                TrueSkillのtauパラメータ。
            draw_probability: float
                TrueSkillのdraw_probabilityパラメータ。
            seed: float
                ランダムシード。
        """
        # 私たちの競技環境
        self._trueskill = TrueSkill(
            mu = mu,
            sigma = sigma,
            beta = beta,
            tau = tau,
            draw_probability = draw_probability
        )
        
        # プレイヤーのプール — それがPlayerコンストラクタに環境を追加する理由
        self.players = { ii: Player(s, self._trueskill) for ii,s in enumerate(skill_dist) }
        self.players.update({ len(self.players)+ii: Player(0, self._trueskill) for ii in range(nb_dummy) })
        
        # ランダムシード
        self._seed = seed
        random.seed(seed)
        
        
    def play_games(self, nb_games: int = 10000):
        """
        ゲームをシミュレーションし、プレイヤーの履歴を更新します。
        
        パラメータ
        ------
            nb_games: int
                プレイするゲームの数。
        """
        
        for i in tqdm(range(nb_games)):
            player_ids = random.sample(range(len(self.players)), 4)  # ランダムに4人のプレイヤーを選びます
            try:
                play_match(
                    [
                        self.players[player_ids[0]],
                        self.players[player_ids[1]]
                    ],
                    [
                        self.players[player_ids[2]],
                        self.players[player_ids[3]]
                    ],
                    env = self._trueskill
                )
            except:
                continue
```

</div>
</details>

In [ ]:
class Competition:
    """
    TrueSkillのコンペティション環境を設定できます。
    """
    
    def __init__(
        self,
        skill_dist: List[float],
        nb_dummy: int,
        mu: float = 600,
        sigma: float = 300,
        beta: float = 150,
        tau: float = 3,
        draw_probability: float = 0.9956,
        seed: float = 42
    ):
        """
        コンストラクター。
        
        パラメータ
        ------
            skill_dist: List[float]
                各「良い」提出のキーワード発見の確率。
            nb_dummy: int
                ダミーボットの数。
            mu: float
                TrueSkillのmuパラメータ。
            sigma: float
                TrueSkillのsigmaパラメータ。
            beta: float
                TrueSkillのbetaパラメータ。
            tau: float
                TrueSkillのtauパラメータ。
            draw_probability: float
                TrueSkillのdraw_probabilityパラメータ。
            seed: float
                ランダムシード。
        """
        # 私たちの競技環境
        self._trueskill = TrueSkill(
            mu = mu,
            sigma = sigma,
            beta = beta,
            tau = tau,
            draw_probability = draw_probability
        )
        
        # プレイヤーのプール — それがPlayerコンストラクタに環境を追加する理由
        self.players = { ii: Player(s, self._trueskill) for ii,s in enumerate(skill_dist) }
        self.players.update({ len(self.players)+ii: Player(0, self._trueskill) for ii in range(nb_dummy) })
        
        # ランダムシード
        self._seed = seed
        random.seed(seed)
        
        
    def play_games(self, nb_games: int = 10000):
        """
        ゲームをシミュレーションし、プレイヤーの履歴を更新します。
        
        パラメータ
        ------
            nb_games: int
                プレイするゲームの数。
        """
        
        for i in tqdm(range(nb_games)):
            player_ids = random.sample(range(len(self.players)), 4)  # ランダムに4人のプレイヤーを選びます
            try:
                play_match(
                    [
                        self.players[player_ids[0]],
                        self.players[player_ids[1]]
                    ],
                    [
                        self.players[player_ids[2]],
                        self.players[player_ids[3]]
                    ],
                    env = self._trueskill
                )
            except:
                continue

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Let's recover the skill distribution we already used.

</div>
<div class="column-right">

# 日本語訳

すでに使用したスキル分布を取得します。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
skill_dist = skills_df["skill"].values
nb_dummy = len(players) - len(skill_dist)
```

</div>
<div class="column-right">

# 日本語訳

```python
skill_dist = skills_df["skill"].values
nb_dummy = len(players) - len(skill_dist)
```

</div>
</details>

In [ ]:
skill_dist = skills_df["skill"].values
nb_dummy = len(players) - len(skill_dist)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# We'll stick to the LLM20 guessed settings, but you can define several competitions to compare them
LLM20 = Competition(skill_dist, nb_dummy)
```

</div>
<div class="column-right">

# 日本語訳

```python
# LLM20のキーワード設定に従いますが、比較のためにいくつかのコンペティションを定義することができます
LLM20 = Competition(skill_dist, nb_dummy)
```

</div>
</details>

In [ ]:
# LLM20のキーワード設定に従いますが、比較のためにいくつかのコンペティションを定義することができます
LLM20 = Competition(skill_dist, nb_dummy)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Expect ~1700 games per second on Kaggle's systems
# Note that the history is not reset, the games add to the previous ones
LLM20.play_games(100000)
```

</div>
<div class="column-right">

# 日本語訳

```python
# Kaggleのシステムで約1700ゲーム／秒を期待してください
# 履歴はリセットされず、ゲームは以前のものに加算されます
LLM20.play_games(100000)
```

</div>
</details>

In [ ]:
# Kaggleのシステムで約1700ゲーム／秒を期待してください
# 履歴はリセットされず、ゲームは以前のものに加算されます
LLM20.play_games(100000)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Plots

Before diving into the results of this simulation, let's keep in mind the distribution of the actual number of games played by each submission in the real world, after two months of competition :

</div>
<div class="column-right">

# 日本語訳

# プロット

シミュレーションの結果に入る前に、実際の数ヶ月間の競技における各提出物のゲームの数の分布を見ておきましょう：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
plt.figure(figsize=(12,6))
last_games_df.groupby("guesser_SubmissionId").size().hist(bins=15)
_ = plt.title("Actual distribution of games played")
_ = plt.xlabel("Number of games played")
_ = plt.ylabel("Number of bots")
```

</div>
<div class="column-right">

# 日本語訳

```python
plt.figure(figsize=(12,6))
last_games_df.groupby("guesser_SubmissionId").size().hist(bins=15)
_ = plt.title("実際のプレイされたゲームの分布")
_ = plt.xlabel("プレイされたゲームの数")
_ = plt.ylabel("ボットの数")
```

</div>
</details>

In [ ]:
plt.figure(figsize=(12,6))
last_games_df.groupby("guesser_SubmissionId").size().hist(bins=15)
_ = plt.title("実際のプレイされたゲームの分布")
_ = plt.xlabel("プレイされたゲームの数")
_ = plt.ylabel("ボットの数")

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
last_games_df["guesser_SubmissionId"].nunique(), (last_games_df.groupby("guesser_SubmissionId").size()>100).sum()
```

</div>
<div class="column-right">

# 日本語訳

```python
last_games_df["guesser_SubmissionId"].nunique(), (last_games_df.groupby("guesser_SubmissionId").size()>100).sum()
```

</div>
</details>

In [ ]:
last_games_df["guesser_SubmissionId"].nunique(), (last_games_df.groupby("guesser_SubmissionId").size()>100).sum()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Some bots played much more games as others because they're much older.  Actually, about half bots played less than 100 games, while some others played more than 500 !  This is an important point that we will just ignore further on, but it's good to have it in mind when comparing to the actual competition.

Now let's make some plots of our simulation to see how this ranking system performs in the long run.

</div>
<div class="column-right">

# 日本語訳

一部のボットは他のボットよりも多くのゲームを行っていますが、実際に大幅に多く、特に古いボットの中にそういったものが存在します。実際、約半数のボットは100ゲーム以下しかプレイしていませんが、一部は500ゲーム以上プレイしています！ これは重要なポイントであり、今後比較する際に無視するつもりですが、これを考慮に入れておくのは良いことです。

これで、シミュレーションの結果をより詳細に確認しましょう。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
fig = plt.figure(figsize=(16,8))
cmap = plt.get_cmap('viridis')
for i in range(len(LLM20.players))[::-1]:  # players sorted by increasing skill, to get the best on top
    pskill = LLM20.players[i].skill
    plt.plot(
        [r.mu for r in LLM20.players[i].history],
        c = cmap(pskill) if pskill>0 else [.6]*4  # bad bots in grey
    )
_ = plt.title("Mu evolution")
_ = plt.xlabel("Number of games played")
_ = plt.ylabel("TrueSkill's mu value")
```

</div>
<div class="column-right">

# 日本語訳

```python
fig = plt.figure(figsize=(16,8))
cmap = plt.get_cmap('viridis')
for i in range(len(LLM20.players))[::-1]:  # スキルの増加に従ってプレイヤーをソートし、最上位を保つ
    pskill = LLM20.players[i].skill
    plt.plot(
        [r.mu for r in LLM20.players[i].history],
        c = cmap(pskill) if pskill>0 else [.6]*4  # 悪いボットは灰色で表示
    )
_ = plt.title("Muの進化")
_ = plt.xlabel("プレイされたゲームの数")
_ = plt.ylabel("TrueSkillのmu値")
```

</div>
</details>

In [ ]:
fig = plt.figure(figsize=(16,8))
cmap = plt.get_cmap('viridis')
for i in range(len(LLM20.players))[::-1]:  # スキルの増加に従ってプレイヤーをソートし、最上位を保つ
    pskill = LLM20.players[i].skill
    plt.plot(
        [r.mu for r in LLM20.players[i].history],
        c = cmap(pskill) if pskill>0 else [.6]*4  # 悪いボットは灰色で表示
    )
_ = plt.title("Muの進化")
_ = plt.xlabel("プレイされたゲームの数")
_ = plt.ylabel("TrueSkillのmu値")

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

The above plot shows that although there is *a trend*, the system fails to distinguish players according to their skill, even after a large number of games.  The gray lines correspond to dummy bots.

Let's have a closer look at the behaviour of the top 10 simulated players :

</div>
<div class="column-right">

# 日本語訳

上記のプロットは、*傾向*はあるものの、システムがプレイヤーをスキルに応じて識別することに失敗していることを示しています。灰色の線はダミーボットに対応しています。

シミュレーションされた上位10人のプレイヤーの挙動にさらに注目しましょう：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
NFIRSTS = 10
fig = plt.figure(figsize=(16,8))
cmap = plt.get_cmap('tab10')
for i in range(NFIRSTS):
    pskill = LLM20.players[i].skill
    plt.plot(
        [r.mu for r in LLM20.players[i].history],
        c = cmap(i),
        label = f"skill = {pskill:.2f}"
    )
_ = plt.legend()
_ = plt.title("Mu evolution of top 10 (simulated) players")
_ = plt.xlabel("Number of games played")
_ = plt.ylabel("TrueSkill's mu value")
```

</div>
<div class="column-right">

# 日本語訳

```python
NFIRSTS = 10
fig = plt.figure(figsize=(16,8))
cmap = plt.get_cmap('tab10')
for i in range(NFIRSTS):
    pskill = LLM20.players[i].skill
    plt.plot(
        [r.mu for r in LLM20.players[i].history],
        c = cmap(i),
        label = f"スキル = {pskill:.2f}"
    )
_ = plt.legend()
_ = plt.title("上位10人（シミュレーションされた）プレイヤーのMuの進化")
_ = plt.xlabel("プレイされたゲームの数")
_ = plt.ylabel("TrueSkillのmu値")
```

</div>
</details>

In [ ]:
NFIRSTS = 10
fig = plt.figure(figsize=(16,8))
cmap = plt.get_cmap('tab10')
for i in range(NFIRSTS):
    pskill = LLM20.players[i].skill
    plt.plot(
        [r.mu for r in LLM20.players[i].history],
        c = cmap(i),
        label = f"スキル = {pskill:.2f}"
    )
_ = plt.legend()
_ = plt.title("上位10人（シミュレーションされた）プレイヤーのMuの進化")
_ = plt.xlabel("プレイされたゲームの数")
_ = plt.ylabel("TrueSkillのmu値")

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

The best players are struggling to keep up with poorer ones…

It may be interesting to compare the evolution of players with the same skill.  We could set a specific competition environment to test that, but I'm more interesting in keeping the most realistic scenario, so let's just check among our current pool of players :

</div>
<div class="column-right">

# 日本語訳

最高のプレイヤーたちは、実際には下位のボットと競争しているようです…

同じスキルを持つプレイヤーの進化を比較することも興味深いかもしれません。特定の競技環境を設定してそれをテストすることもできますが、私は現実的なシナリオを保ちたいので、現在のプレイヤープールの中から見てみましょう：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from collections import Counter
```

</div>
<div class="column-right">

# 日本語訳

```python
from collections import Counter
```

</div>
</details>

In [ ]:
from collections import Counter

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
duplicated_skills = Counter(skill_dist[:15]).most_common(2)
duplicated_skills
```

</div>
<div class="column-right">

# 日本語訳

```python
duplicated_skills = Counter(skill_dist[:15]).most_common(2)
duplicated_skills
```

</div>
</details>

In [ ]:
duplicated_skills = Counter(skill_dist[:15]).most_common(2)
duplicated_skills

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

We have two couples of interesting candidates among the 15 best players.  Let's check how they performed over time.

</div>
<div class="column-right">

# 日本語訳

上位15人のプレイヤーの中に、興味深い候補が2組存在します。それらのパフォーマンスを時系列で確認してみましょう。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
SHOW_ALL = True
fig = plt.figure(figsize=(16,8))
cmap = plt.get_cmap('tab10')
gray = [.05] * 4
if SHOW_ALL:
    for i in range(len(LLM20.players)):
        plt.plot(
            [r.mu for r in LLM20.players[i].history],
            c = gray
        )
for i in range(20):
    pskill = LLM20.players[i].skill
    if pskill == duplicated_skills[0][0]:
        color = cmap(0)
    elif pskill == duplicated_skills[1][0]:
        color = cmap(1)
    else:
        continue
    plt.plot(
        [r.mu for r in LLM20.players[i].history],
        c = color,
        label = f"skill = {pskill:.2f}"
    )
_ = plt.legend()
_ = plt.title("Mu evolution of players with identical skills")
_ = plt.xlabel("Number of games played")
_ = plt.ylabel("TrueSkill's mu value")
```

</div>
<div class="column-right">

# 日本語訳

```python
SHOW_ALL = True
fig = plt.figure(figsize=(16,8))
cmap = plt.get_cmap('tab10')
gray = [.05] * 4
if SHOW_ALL:
    for i in range(len(LLM20.players)):
        plt.plot(
            [r.mu for r in LLM20.players[i].history],
            c = gray
        )
for i in range(20):
    pskill = LLM20.players[i].skill
    if pskill == duplicated_skills[0][0]:
        color = cmap(0)
    elif pskill == duplicated_skills[1][0]:
        color = cmap(1)
    else:
        continue
    plt.plot(
        [r.mu for r in LLM20.players[i].history],
        c = color,
        label = f"スキル = {pskill:.2f}"
    )
_ = plt.legend()
_ = plt.title("同一スキルを持つプレイヤーのMuの進化")
_ = plt.xlabel("プレイされたゲームの数")
_ = plt.ylabel("TrueSkillのmu値")
```

</div>
</details>

In [ ]:
SHOW_ALL = True
fig = plt.figure(figsize=(16,8))
cmap = plt.get_cmap('tab10')
gray = [.05] * 4
if SHOW_ALL:
    for i in range(len(LLM20.players)):
        plt.plot(
            [r.mu for r in LLM20.players[i].history],
            c = gray
        )
for i in range(20):
    pskill = LLM20.players[i].skill
    if pskill == duplicated_skills[0][0]:
        color = cmap(0)
    elif pskill == duplicated_skills[1][0]:
        color = cmap(1)
    else:
        continue
    plt.plot(
        [r.mu for r in LLM20.players[i].history],
        c = color,
        label = f"スキル = {pskill:.2f}"
    )
_ = plt.legend()
_ = plt.title("同一スキルを持つプレイヤーのMuの進化")
_ = plt.xlabel("プレイされたゲームの数")
_ = plt.ylabel("TrueSkillのmu値")

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Although there are some less/more skilled competitors inbetween, hat's not that bad, considering that @c-number, who owns currently the first place of the actual leaderboard, reported that their three identical submissions [were spread on 60 positions](https://www.kaggle.com/competitions/llm-20-questions/discussion/520928), with a score varying from $\mu=1143$ to… $\mu=767$ !!!

# Comparing with the leaderboard

Let's check how our simulated leaderboard compares with the actual one :

</div>
<div class="column-right">

# 日本語訳

同じスキルを持つ競技者の中に少し上下の差が見られますが、@c-numberが実際のリーダーボードで1位を占めていることを考慮すると、同じ提出物が60位置で分散して、スコアは$\mu=1143$から… $\mu=767$にまで変動すると報告されましたから！

# リーダーボードとの比較

シミュレーションされたリーダーボードが実際のリーダーボードとどのように比較されるかを確認しましょう：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
lb = pd.read_csv("/kaggle/input/llm-20-lb-2024-07-14/llm-20-questions-publicleaderboard-2024-07-14.csv")
```

</div>
<div class="column-right">

# 日本語訳

```python
lb = pd.read_csv("/kaggle/input/llm-20-lb-2024-07-14/llm-20-questions-publicleaderboard-2024-07-14.csv")
```

</div>
</details>

In [ ]:
lb = pd.read_csv("/kaggle/input/llm-20-lb-2024-07-14/llm-20-questions-publicleaderboard-2024-07-14.csv")

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
plt.figure(figsize=(12,6))
_ = plt.hist(
    lb.Score,
    bins = 100,
    range = (300, 1200),
    log = True,
    label = "Leaderboard"
)
_ = plt.hist(
    [ p.history[-1].mu for p in LLM20.players.values() ],
    bins = 100,
    range = (300, 1200),
    alpha = .5,
    log = True,
    label = "Simulation"
)
_ = plt.title("Played games corrected leaderboard distribution")
_ = plt.xlabel("TrueSkill mu")
_ = plt.ylabel("Number of bots")
_ = plt.legend()
```

</div>
<div class="column-right">

# 日本語訳

```python
plt.figure(figsize=(12,6))
_ = plt.hist(
    lb.Score,
    bins = 100,
    range = (300, 1200),
    log = True,
    label = "リーダーボード"
)
_ = plt.hist(
    [ p.history[-1].mu for p in LLM20.players.values() ],
    bins = 100,
    range = (300, 1200),
    alpha = .5,
    log = True,
    label = "シミュレーション"
)
_ = plt.title("プレイゲームによって修正されたリーダーボード分布")
_ = plt.xlabel("TrueSkill mu")
_ = plt.ylabel("ボットの数")
_ = plt.legend()
```

</div>
</details>

In [ ]:
plt.figure(figsize=(12,6))
_ = plt.hist(
    lb.Score,
    bins = 100,
    range = (300, 1200),
    log = True,
    label = "リーダーボード"
)
_ = plt.hist(
    [ p.history[-1].mu for p in LLM20.players.values() ],
    bins = 100,
    range = (300, 1200),
    alpha = .5,
    log = True,
    label = "シミュレーション"
)
_ = plt.title("プレイゲームによって修正されたリーダーボード分布")
_ = plt.xlabel("TrueSkill mu")
_ = plt.ylabel("ボットの数")
_ = plt.legend()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

That's far from perfect, but not too bad either.  Part of the differences can be explained by the different number of games played, but it's not enough, see the corrected comparison :

</div>
<div class="column-right">

# 日本語訳

完璧とは言えませんが、そこそこ良い結果です。差異の一部は、ゲームの数の違いによって説明可以險が、繊細には不十分です。修正された比較を見てみましょう：

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
plt.figure(figsize=(12,6))
played_dist = last_games_df.groupby("guesser_SubmissionId").size().values
_ = plt.hist(
    lb.Score,
    bins = 100,
    range = (300, 1200),
    log = True,
    label = "Leaderboard"
)
_ = plt.hist(
    [
        p.history[min(len(p.history)-1, random.choice(played_dist))].mu  # we pick the "last game" number from the played games distribution
        for p in LLM20.players.values()
    ],
    bins = 100,
    range = (300, 1200),
    alpha = .5,
    log = True,
    label = "Simulation"
)
_ = plt.title("Played games corrected leaderboard distribution")
_ = plt.xlabel("TrueSkill mu")
_ = plt.ylabel("Number of bots")
_ =  plt.legend()
```

</div>
<div class="column-right">

# 日本語訳

```python
plt.figure(figsize=(12,6))
played_dist = last_games_df.groupby("guesser_SubmissionId").size().values
_ = plt.hist(
    lb.Score,
    bins = 100,
    range = (300, 1200),
    log = True,
    label = "リーダーボード"
)
_ = plt.hist(
    [
        p.history[min(len(p.history)-1, random.choice(played_dist))].mu  # プレイされたゲームの分布から「最後のゲーム」数を選びます
        for p in LLM20.players.values()
    ],
    bins = 100,
    range = (300, 1200),
    alpha = .5,
    log = True,
    label = "シミュレーション"
)
_ = plt.title("プレイゲームによって修正されたリーダーボード分布")
_ = plt.xlabel("TrueSkill mu")
_ = plt.ylabel("ボットの数")
_ =  plt.legend()
```

</div>
</details>

In [ ]:
plt.figure(figsize=(12,6))
played_dist = last_games_df.groupby("guesser_SubmissionId").size().values
_ = plt.hist(
    lb.Score,
    bins = 100,
    range = (300, 1200),
    log = True,
    label = "リーダーボード"
)
_ = plt.hist(
    [
        p.history[min(len(p.history)-1, random.choice(played_dist))].mu  # プレイされたゲームの分布から「最後のゲーム」数を選びます
        for p in LLM20.players.values()
    ],
    bins = 100,
    range = (300, 1200),
    alpha = .5,
    log = True,
    label = "シミュレーション"
)
_ = plt.title("プレイゲームによって修正されたリーダーボード分布")
_ = plt.xlabel("TrueSkill mu")
_ = plt.ylabel("ボットの数")
_ =  plt.legend()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Another point is the skill distribution being shifted towards higher values, but again it doesn't suffice to explain the difference, so I think maybe the value we set for $\beta$ or $\tau$ may be off somehow.  There is also the possibility that the parameters were adjusted along the competition.

Anyway, from my tests I strongly doubt it would change the outcome of this simulation.

# Comparing with the top10 games history

I manually scrapped the game history of the leaderboard's top 10 positions and processed the data into a dictionary which can be found as a pickle in the [LLM-20-top10-LB-matches](https://www.kaggle.com/datasets/gguillard/llm-20-top10-lb-matches) dataset.

</div>
<div class="column-right">

# 日本語訳

もう一つの点は、スキル分布が高い値にシフトしていることですが、やはりそれだけでは違いを説明するには不十分です。したがって、$\beta$や$\tau$の値が何らかの形でずれている可能性があると思います。また、競技を通じてパラメータが調整された可能性もあります。

いずれにせよ、私のテストからはこのシミュレーションの結果が変わるとは思えません。

# 上位10ゲームの履歴との比較

私は手動でリーダーボードの上位10位のゲーム履歴をスクレイピングし、そのデータを辞書に処理しました。この辞書は、[LLM-20-top10-LB-matches](https://www.kaggle.com/datasets/gguillard/llm-20-top10-lb-matches)データセットのピクル形式で見つけることができます。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import pickle
with open("/kaggle/input/llm-20-top10-lb-matches/matches.pickle", "rb") as f:
    top10_games_dict = pickle.load(f)
```

</div>
<div class="column-right">

# 日本語訳

```python
import pickle
with open("/kaggle/input/llm-20-top10-lb-matches/matches.pickle", "rb") as f:
    top10_games_dict = pickle.load(f)
```

</div>
</details>

In [ ]:
import pickle
with open("/kaggle/input/llm-20-top10-lb-matches/matches.pickle", "rb") as f:
    top10_games_dict = pickle.load(f)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

How does their evolution compares with our simulation ?

</div>
<div class="column-right">

# 日本語訳

彼らの進化は私たちのシミュレーションとどのように比較されるのでしょうか？


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
plt.figure(figsize=(12,6))
gray = [.05] * 4
if SHOW_ALL:
    for i in range(len(LLM20.players)):
        plt.plot(
            [r.mu for r in LLM20.players[i].history[:200]],
            c = gray
        )
for kaggler, df in top10_games_dict.items():
    _ = plt.plot(df[kaggler], label = kaggler)
_ = plt.title("Top 10 leaderboard mu evolution (background = simulation)")
_ = plt.xlabel("Number of games played")
_ = plt.ylabel("TrueSkill's mu value")
_ = plt.legend()
```

</div>
<div class="column-right">

# 日本語訳

```python
plt.figure(figsize=(12,6))
gray = [.05] * 4
if SHOW_ALL:
    for i in range(len(LLM20.players)):
        plt.plot(
            [r.mu for r in LLM20.players[i].history[:200]],
            c = gray
        )
for kaggler, df in top10_games_dict.items():
    _ = plt.plot(df[kaggler], label = kaggler)
_ = plt.title("上位10名のリーダーボードのmuの進化（背景 = シミュレーション）")
_ = plt.xlabel("プレイしたゲームの数")
_ = plt.ylabel("TrueSkillのmu値")
_ = plt.legend()
```

</div>
</details>

In [ ]:
plt.figure(figsize=(12,6))
gray = [.05] * 4
if SHOW_ALL:
    for i in range(len(LLM20.players)):
        plt.plot(
            [r.mu for r in LLM20.players[i].history[:200]],
            c = gray
        )
for kaggler, df in top10_games_dict.items():
    _ = plt.plot(df[kaggler], label = kaggler)
_ = plt.title("上位10名のリーダーボードのmuの進化（背景 = シミュレーション）")
_ = plt.xlabel("プレイしたゲームの数")
_ = plt.ylabel("TrueSkillのmu値")
_ = plt.legend()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

The point of this plot is just to show that the leaderboard behaviour is on par with our simulation.  Note the impressive remontada from SpiralTip…

# Team matching

There is an important point that we didn't take into account in all this process.  Quoting the competition overview : 

> Each submission will play episodes (games) against other bots on the leaderboard that have a similar skill rating.

AFAICS, the TrueSkill library doesn't provide a convenient function to draw players according to their skill rating.  Is it worth bothering about this point ?  Since we have the top 10 players games history, we can check how they were paired along their $\mu$ evolution :

</div>
<div class="column-right">

# 日本語訳

このプロットの目的は、リーダーボードの挙動が私たちのシミュレーションと一致していることを示すことです。SpiralTipの印象的な復活にも注目してください…

# チームのマッチング

このプロセスで考慮しなかった重要な点があります。競技の概要を引用します：

> 各提出物は、同等のスキルレーティングを持つ他のボットとエピソード（ゲーム）をプレイします。

私の見解では、TrueSkillライブラリにはスキルレーティングに従ってプレイヤーを引き出す便利な関数が提供されていません。この点についてこだわる価値はあるのでしょうか？ 上位10プレイヤーのゲーム履歴があるので、彼らが$\mu$の進化に沿ってどのようにペアリングされたかを確認できます：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
plt.figure(figsize=(12,6))
for kaggler, df in top10_games_dict.items():
    _ = plt.scatter(df[kaggler], df["Op1"], s = 1, label = kaggler)
    _ = plt.scatter(df[kaggler], df["Op2"], s = 1)
    _ = plt.scatter(df[kaggler], df["Op3"], s = 1)
_ = plt.legend()
```

</div>
<div class="column-right">

# 日本語訳

```python
plt.figure(figsize=(12,6))
for kaggler, df in top10_games_dict.items():
    _ = plt.scatter(df[kaggler], df["Op1"], s = 1, label = kaggler)
    _ = plt.scatter(df[kaggler], df["Op2"], s = 1)
    _ = plt.scatter(df[kaggler], df["Op3"], s = 1)
_ = plt.legend()
```

</div>
</details>

In [ ]:
plt.figure(figsize=(12,6))
for kaggler, df in top10_games_dict.items():
    _ = plt.scatter(df[kaggler], df["Op1"], s = 1, label = kaggler)
    _ = plt.scatter(df[kaggler], df["Op2"], s = 1)
    _ = plt.scatter(df[kaggler], df["Op3"], s = 1)
_ = plt.legend()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Although there is some trend, it is not obvious how to infer a simulation function from that.  Furthermore, since the high scoring bots can still be paired with dummy bots, their ratings can still be degraded (although it's somehow mitigated by their smaller $\sigma$).

But most of all, this doesn't address (and on the contrary enforce) the "[pit of dumbness](https://www.kaggle.com/competitions/llm-20-questions/discussion/514628#2889458)" issue, as @loh-maa named it very acurately.

# Conclusion

Many things are probably missing in this simulation.  In particular, the fact that there are new submissions over time, the fact that the game frequency decreases over time for each player (maybe in relation to sigma ?), or the fact that to a certain extent, players are matched against players of a similar rank.  I don't think any of these "feature" would change my conclusions, though.  It is clear to me that in its current state, the ranking system of the LLM 20 Questions competition gives more room to chance than to performance.

# Playground

If you wanna just simulate a TrueSkill environment without bothering (re)reading everything above, here is everything you need.

</div>
<div class="column-right">

# 日本語訳

トレンドは見られますが、そこからシミュレーション関数を推測するのは明確ではありません。さらに、高得点のボットがダミーボットとペアにされる可能性があるため、そのレーティングが低下する可能性があります（ただし、それは彼らの小さな$\sigma$によってある程度緩和されています）。

しかし、何よりも、この点は（逆に「愚かさの落とし穴」問題を助長し）解決されていません。@loh-maaが非常に的確に名付けた問題です。

# 結論

このシミュレーションにはおそらく多くの欠陥があるでしょう。特に、新しい提出物が時間とともに増えること、各プレイヤーのゲームの頻度が時間とともに減少すること（おそらくσに関連しているかもしれません）、プレイヤーが同様のランキングのプレイヤーと対戦することがある程度あるという事実です。しかし、これらの「特徴」が私の結論を変えることはないと思います。現状のままでは、LLM 20 Questionsコンペティションのランキングシステムは、パフォーマンスよりも偶然の要素を多く含んでいることは明らかです。

# プレイグラウンド

もし、上記の内容を（再）読まなくてもTrueSkill環境をシミュレートしたい場合は、必要なすべてのコードがあります。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install trueskill
```

</div>
<div class="column-right">

# 日本語訳

```python
!pip install trueskill
```

</div>
</details>

In [ ]:
!pip install trueskill

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import random
from typing import List, Optional, Tuple
import warnings

from tqdm import tqdm
from trueskill import TrueSkill
```

</div>
<div class="column-right">

# 日本語訳

```python
import random
from typing import List, Optional, Tuple
import warnings

from tqdm import tqdm
from trueskill import TrueSkill
```

</div>
</details>

In [ ]:
import random
from typing import List, Optional, Tuple
import warnings

from tqdm import tqdm
from trueskill import TrueSkill

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
class Player:
    def __init__(
        self,
        skill: float,
        env: Optional[TrueSkill] = None  # ignore this for now, this will be useful later
    ):
        self.skill = skill
        self.rating = Rating() if env is None else env.Rating()  # start with environment default values, i.e. (600,300)
        self.history = []
        
    def update(self, new_rating):
        self.history.append(self.rating)
        self.rating = new_rating
```

</div>
<div class="column-right">

# 日本語訳

```python
class Player:
    def __init__(
        self,
        skill: float,
        env: Optional[TrueSkill] = None  # 今は無視しますが、後で使えるようにするためです
    ):
        self.skill = skill
        self.rating = Rating() if env is None else env.Rating()  # 環境のデフォルト値（つまり600,300）で開始
        self.history = []
        
    def update(self, new_rating):
        self.history.append(self.rating)
        self.rating = new_rating
```

</div>
</details>

In [ ]:
class Player:
    def __init__(
        self,
        skill: float,
        env: Optional[TrueSkill] = None  # 今は無視しますが、後で使えるようにするためです
    ):
        self.skill = skill
        self.rating = Rating() if env is None else env.Rating()  # 環境のデフォルト値（つまり600,300）で開始
        self.history = []
        
    def update(self, new_rating):
        self.history.append(self.rating)
        self.rating = new_rating

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def play_match(
    teamA: Tuple[Player, Player],
    teamB: Tuple[Player, Player],
    env: Optional[TrueSkill],
    debug: bool = False
):
    """
    Plays a match between two teams.
    
    Params
    ------
        teamA: Tuple[Player, Player]
            A team of two players.
        teamB: Tuple[Player, Player]
            Another team of two players.
        debug: boolean
            A flag for debugging.
    """
    if debug:
        print("Players skills :", [p.skill for p in teamA+teamB])
        print("Players ratings :", [tuple(p.rating for p in teamA), tuple(p.rating for p in teamB)])
    # for each team, if any player is a dummy bot, the team fails to find the keyword
    # else the team finds the keyword if its guesser's skill is higher than a random number
    # (beware of the reversed logic in the code)
    ranks = [
        1 if teamA[0].skill * teamA[1].skill == 0 or random.random() > teamA[0].skill else 0,
        1 if teamB[0].skill * teamB[1].skill == 0 or random.random() > teamB[0].skill else 0
    ]
    
    # new ratings are the results of the game
    new_ratings = env.rate(
        [
            [ p.rating for p in teamA ],
            [ p.rating for p in teamB ]
        ],
        ranks
    )
    
    # update all players' ratings
    for p,r in zip(teamA, new_ratings[0]):
        p.update(r)
    for p,r in zip(teamB, new_ratings[1]):
        p.update(r)
        
    if debug:
        print("Game outcome :", ranks)
        print("New ratings :", new_ratings)
    
    return
```

</div>
<div class="column-right">

# 日本語訳

```python
def play_match(
    teamA: Tuple[Player, Player],
    teamB: Tuple[Player, Player],
    env: Optional[TrueSkill],
    debug: bool = False
):
    """
    2チーム間の試合を行います。
    
    パラメータ
    ------
        teamA: Tuple[Player, Player]
            2人のプレイヤーからなるチーム。
        teamB: Tuple[Player, Player]
            もう一つの2人のプレイヤーからなるチーム。
        debug: boolean
            デバッグ用フラグ。
    """
    if debug:
        print("プレイヤーのスキル　:", [p.skill for p in teamA+teamB])
        print("プレイヤーのレーティング　:", [tuple(p.rating for p in teamA), tuple(p.rating for p in teamB)])
    # 各チームについて、どのプレイヤーがダミーボットかを判定し、ボットがキーワードを見つけられなければ失敗
    # それ以外の場合、チームの目標スキルがランダムな数よりも高ければキーワードを見つけられる
    # （影に逆の論理があることに注意）
    ranks = [
        1 if teamA[0].skill * teamA[1].skill == 0 or random.random() > teamA[0].skill else 0,
        1 if teamB[0].skill * teamB[1].skill == 0 or random.random() > teamB[0].skill else 0
    ]
    
    # 新しいレーティングはゲームの結果
    new_ratings = env.rate(
        [
            [ p.rating for p in teamA ],
            [ p.rating for p in teamB ]
        ],
        ranks
    )
    
    # すべてのプレイヤーのレーティングを更新
    for p,r in zip(teamA, new_ratings[0]):
        p.update(r)
    for p,r in zip(teamB, new_ratings[1]):
        p.update(r)
        
    if debug:
        print("ゲームの結果　:", ranks)
        print("新しいレーティング　:", new_ratings)
    
    return
```

</div>
</details>

In [ ]:
def play_match(
    teamA: Tuple[Player, Player],
    teamB: Tuple[Player, Player],
    env: Optional[TrueSkill],
    debug: bool = False
):
    """
    2チーム間の試合を行います。
    
    パラメータ
    ------
        teamA: Tuple[Player, Player]
            2人のプレイヤーからなるチーム。
        teamB: Tuple[Player, Player]
            もう一つの2人のプレイヤーからなるチーム。
        debug: boolean
            デバッグ用フラグ。
    """
    if debug:
        print("プレイヤーのスキル　:", [p.skill for p in teamA+teamB])
        print("プレイヤーのレーティング　:", [tuple(p.rating for p in teamA), tuple(p.rating for p in teamB)])
    # 各チームについて、どのプレイヤーがダミーボットかを判定し、ボットがキーワードを見つけられなければ失敗
    # それ以外の場合、チームの目標スキルがランダムな数よりも高ければキーワードを見つけられる
    # （影に逆の論理があることに注意）
    ranks = [
        1 if teamA[0].skill * teamA[1].skill == 0 or random.random() > teamA[0].skill else 0,
        1 if teamB[0].skill * teamB[1].skill == 0 or random.random() > teamB[0].skill else 0
    ]
    
    # 新しいレーティングはゲームの結果
    new_ratings = env.rate(
        [
            [ p.rating for p in teamA ],
            [ p.rating for p in teamB ]
        ],
        ranks
    )
    
    # すべてのプレイヤーのレーティングを更新
    for p,r in zip(teamA, new_ratings[0]):
        p.update(r)
    for p,r in zip(teamB, new_ratings[1]):
        p.update(r)
        
    if debug:
        print("ゲームの結果　:", ranks)
        print("新しいレーティング　:", new_ratings)
    
    return

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
class Competition:
    """
    A configurable TrueSkill competition environment.
    """
    
    def __init__(
        self,
        skill_dist: List[float],
        nb_dummy: int,
        mu: float = 600,
        sigma: float = 300,
        beta: float = 150,
        tau: float = 3,
        draw_probability: float = 0.9956,
        seed: float = 42,
        debug: bool = False
    ):
        """
        Constructor.
        
        Params
        ------
            skill_dist: List[float]
                The probability to find the keyword, for each "good" submission.
            nb_dummy: int
                The number of dummy bots
            mu: float
                The TrueSkill mu parameter.
            sigma: float
                The TrueSkill sigma parameter.
            beta: float
                The TrueSkill beta parameter.
            tau: float
                The TrueSkill tau parameter.
            draw_probability: float
                The TrueSkill draw_probability parameter.
            seed: float
                A random seed.
            debug: bool
                A debugging flag.
        """
        # our competition environment
        self._trueskill = TrueSkill(
            mu = mu,
            sigma = sigma,
            beta = beta,
            tau = tau,
            draw_probability = draw_probability
        )
        
        # our pool of players — and that's why we needed an "env" parameters for the Player constructor
        self.players = { ii: Player(s, self._trueskill) for ii,s in enumerate(skill_dist) }
        self.players.update({ len(self.players)+ii: Player(0, self._trueskill) for ii in range(nb_dummy) })
        
        # the random seed
        self._seed = seed
        random.seed(seed)
        
        self.debug = debug
        
        
    def play_games(self, nb_games: int = 10000):
        """
        Simulate games and update the players history.
        
        Params
        ------
            nb_games: int
                The number of games to play.
        """
        
        for i in tqdm(range(nb_games)):
            player_ids = random.sample(range(len(self.players)), 4)  # let's pick 4 players randomly
            try:
                play_match(
                    [
                        self.players[player_ids[0]],
                        self.players[player_ids[1]]
                    ],
                    [
                        self.players[player_ids[2]],
                        self.players[player_ids[3]]
                    ],
                    env = self._trueskill,
                    debug = self.debug
                )
            except Exception as e:
                warnings.warn(f"Caught exception {e}")
                continue
```

</div>
<div class="column-right">

# 日本語訳

```python
class Competition:
    """
    TrueSkillのコンペティション環境を設定できます。
    """
    
    def __init__(
        self,
        skill_dist: List[float],
        nb_dummy: int,
        mu: float = 600,
        sigma: float = 300,
        beta: float = 150,
        tau: float = 3,
        draw_probability: float = 0.9956,
        seed: float = 42,
        debug: bool = False
    ):
        """
        コンストラクター。
        
        パラメータ
        ------
            skill_dist: List[float]
                各「良い」提出のキーワード発見の確率。
            nb_dummy: int
                ダミーボットの数。
            mu: float
                TrueSkillのmuパラメータ。
            sigma: float
                TrueSkillのsigmaパラメータ。
            beta: float
                TrueSkillのbetaパラメータ。
            tau: float
                TrueSkillのtauパラメータ。
            draw_probability: float
                TrueSkillのdraw_probabilityパラメータ。
            seed: float
                ランダムシード。
            debug: bool
                デバッグ用フラグ。
        """
        # 私たちの競技環境
        self._trueskill = TrueSkill(
            mu = mu,
            sigma = sigma,
            beta = beta,
            tau = tau,
            draw_probability = draw_probability
        )
        
        # プレイヤーのプール — それがPlayerコンストラクタに環境を追加する理由
        self.players = { ii: Player(s, self._trueskill) for ii,s in enumerate(skill_dist) }
        self.players.update({ len(self.players)+ii: Player(0, self._trueskill) for ii in range(nb_dummy) })
        
        # ランダムシード
        self._seed = seed
        random.seed(seed)
        
        self.debug = debug
        
        
    def play_games(self, nb_games: int = 10000):
        """
        ゲームをシミュレーションし、プレイヤーの履歴を更新します。
        
        パラメータ
        ------
            nb_games: int
                プレイするゲームの数。
        """
        
        for i in tqdm(range(nb_games)):
            player_ids = random.sample(range(len(self.players)), 4)  # ランダムに4人のプレイヤーを選びます
            try:
                play_match(
                    [
                        self.players[player_ids[0]],
                        self.players[player_ids[1]]
                    ],
                    [
                        self.players[player_ids[2]],
                        self.players[player_ids[3]]
                    ],
                    env = self._trueskill,
                    debug = self.debug
                )
            except Exception as e:
                warnings.warn(f"例外が発生しました: {e}")
                continue
```

</div>
</details>

In [ ]:
class Competition:
    """
    TrueSkillのコンペティション環境を設定できます。
    """
    
    def __init__(
        self,
        skill_dist: List[float],
        nb_dummy: int,
        mu: float = 600,
        sigma: float = 300,
        beta: float = 150,
        tau: float = 3,
        draw_probability: float = 0.9956,
        seed: float = 42,
        debug: bool = False
    ):
        """
        コンストラクター。
        
        パラメータ
        ------
            skill_dist: List[float]
                各「良い」提出のキーワード発見の確率。
            nb_dummy: int
                ダミーボットの数。
            mu: float
                TrueSkillのmuパラメータ。
            sigma: float
                TrueSkillのsigmaパラメータ。
            beta: float
                TrueSkillのbetaパラメータ。
            tau: float
                TrueSkillのtauパラメータ。
            draw_probability: float
                TrueSkillのdraw_probabilityパラメータ。
            seed: float
                ランダムシード。
            debug: bool
                デバッグ用フラグ。
        """
        # 私たちの競技環境
        self._trueskill = TrueSkill(
            mu = mu,
            sigma = sigma,
            beta = beta,
            tau = tau,
            draw_probability = draw_probability
        )
        
        # プレイヤーのプール — それがPlayerコンストラクタに環境を追加する理由
        self.players = { ii: Player(s, self._trueskill) for ii,s in enumerate(skill_dist) }
        self.players.update({ len(self.players)+ii: Player(0, self._trueskill) for ii in range(nb_dummy) })
        
        # ランダムシード
        self._seed = seed
        random.seed(seed)
        
        self.debug = debug
        
        
    def play_games(self, nb_games: int = 10000):
        """
        ゲームをシミュレーションし、プレイヤーの履歴を更新します。
        
        パラメータ
        ------
            nb_games: int
                プレイするゲームの数。
        """
        
        for i in tqdm(range(nb_games)):
            player_ids = random.sample(range(len(self.players)), 4)  # ランダムに4人のプレイヤーを選びます
            try:
                play_match(
                    [
                        self.players[player_ids[0]],
                        self.players[player_ids[1]]
                    ],
                    [
                        self.players[player_ids[2]],
                        self.players[player_ids[3]]
                    ],
                    env = self._trueskill,
                    debug = self.debug
                )
            except Exception as e:
                warnings.warn(f"例外が発生しました: {e}")
                continue

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
nb_players = 10  # number of actual players
nb_dummy_players = 10  # number of dummy ("bad") bots
skill_distribution = [ random.random() for i in range(nb_players)]  # skill distribution
```

</div>
<div class="column-right">

# 日本語訳

```python
nb_players = 10  # 実際のプレイヤー数
nb_dummy_players = 10  # ダミー（「悪い」）ボットの数
skill_distribution = [ random.random() for i in range(nb_players)]  # スキル分布
```

</div>
</details>

In [ ]:
nb_players = 10  # 実際のプレイヤー数
nb_dummy_players = 10  # ダミー（「悪い」）ボットの数
skill_distribution = [ random.random() for i in range(nb_players)]  # スキル分布

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# you can define several competitions to compare them
C1 = Competition(skill_distribution, nb_dummy_players)
C2 = Competition(skill_distribution, nb_dummy_players, mu = 500, sigma = 100, beta = 200, tau = 10, draw_probability = .8, seed = 123)
```

</div>
<div class="column-right">

# 日本語訳

```python
# さまざまなコンペティションを定義して比較することができます
C1 = Competition(skill_distribution, nb_dummy_players)
C2 = Competition(skill_distribution, nb_dummy_players, mu = 500, sigma = 100, beta = 200, tau = 10, draw_probability = .8, seed = 123)
```

</div>
</details>

In [ ]:
# さまざまなコンペティションを定義して比較することができます
C1 = Competition(skill_distribution, nb_dummy_players)
C2 = Competition(skill_distribution, nb_dummy_players, mu = 500, sigma = 100, beta = 200, tau = 10, draw_probability = .8, seed = 123)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Expect ~1700 games per second on Kaggle's systems
# Note that the history is not reset, the games add to the previous ones
C1.play_games(10000)
C2.play_games(10000)
```

</div>
<div class="column-right">

# 日本語訳

```python
# Kaggleのシステムで約1700ゲーム／秒を期待してください
# 履歴はリセットされず、ゲームは以前のものに加算されます
C1.play_games(10000)
C2.play_games(10000)
```

</div>
</details>

In [ ]:
# Kaggleのシステムで約1700ゲーム／秒を期待してください
# 履歴はリセットされず、ゲームは以前のものに加算されます
C1.play_games(10000)
C2.play_games(10000)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import matplotlib.pyplot as plt
```

</div>
<div class="column-right">

# 日本語訳

```python
import matplotlib.pyplot as plt
```

</div>
</details>

In [ ]:
import matplotlib.pyplot as plt

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
plt.plot([ r.mu for r in C1.players[0].history])
```

</div>
<div class="column-right">

# 日本語訳

```python
plt.plot([ r.mu for r in C1.players[0].history])
```

</div>
</details>

In [ ]:
plt.plot([ r.mu for r in C1.players[0].history])

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Variability over seed

</div>
<div class="column-right">

# 日本語訳

# シードによる変動


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
C = [Competition(skill_dist, nb_dummy, seed=i) for i in range(4)]
```

</div>
<div class="column-right">

# 日本語訳

```python
C = [Competition(skill_dist, nb_dummy, seed=i) for i in range(4)]
```

</div>
</details>

In [ ]:
C = [Competition(skill_dist, nb_dummy, seed=i) for i in range(4)]

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
for comp in C:
    comp.play_games(50000)
```

</div>
<div class="column-right">

# 日本語訳

```python
for comp in C:
    comp.play_games(50000)
```

</div>
</details>

In [ ]:
for comp in C:
    comp.play_games(50000)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def plot_first_n(comp, nfirsts, a):
    #fig = plt.figure(figsize=(16,8))
    cmap = plt.get_cmap('tab10')
    for i in range(nfirsts):
        pskill = comp.players[i].skill
        a.plot(
            [r.mu for r in comp.players[i].history],
            c = cmap(i),
            label = f"skill = {pskill:.2f}"
        )
    _ = a.legend()
    _ = a.set_title("Mu evolution of top 10 (simulated) players")
    _ = a.set_xlabel("Number of games played")
    _ = a.set_ylabel("TrueSkill's mu value")
```

</div>
<div class="column-right">

# 日本語訳

```python
def plot_first_n(comp, nfirsts, a):
    #fig = plt.figure(figsize=(16,8))
    cmap = plt.get_cmap('tab10')
    for i in range(nfirsts):
        pskill = comp.players[i].skill
        a.plot(
            [r.mu for r in comp.players[i].history],
            c = cmap(i),
            label = f"スキル = {pskill:.2f}"
        )
    _ = a.legend()
    _ = a.set_title("上位10人（シミュレーションされた）プレイヤーのMuの進化")
    _ = a.set_xlabel("プレイされたゲームの数")
    _ = a.set_ylabel("TrueSkillのmu値")
```

</div>
</details>

In [ ]:
def plot_first_n(comp, nfirsts, a):
    #fig = plt.figure(figsize=(16,8))
    cmap = plt.get_cmap('tab10')
    for i in range(nfirsts):
        pskill = comp.players[i].skill
        a.plot(
            [r.mu for r in comp.players[i].history],
            c = cmap(i),
            label = f"スキル = {pskill:.2f}"
        )
    _ = a.legend()
    _ = a.set_title("上位10人（シミュレーションされた）プレイヤーのMuの進化")
    _ = a.set_xlabel("プレイされたゲームの数")
    _ = a.set_ylabel("TrueSkillのmu値")

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
fig, ax = plt.subplots(nrows=4, ncols=1, figsize=(16,16), constrained_layout=True)
for i,comp in enumerate(C):
    #plt.axes(ax[i])
    plot_first_n(comp, 5, ax[i])
```

</div>
<div class="column-right">

# 日本語訳

```python
fig, ax = plt.subplots(nrows=4, ncols=1, figsize=(16,16), constrained_layout=True)
for i,comp in enumerate(C):
    #plt.axes(ax[i])
    plot_first_n(comp, 5, ax[i])
```

</div>
</details>

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=1, figsize=(16,16), constrained_layout=True)
for i,comp in enumerate(C):
    #plt.axes(ax[i])
    plot_first_n(comp, 5, ax[i])

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
for comp in C:
    scores = {p: v.rating.mu for p,v in comp.players.items()}
    leaderboard = {k:v for k,v in sorted(scores.items(), key=lambda item: item[1], reverse = True)}
    print("Leaderboard")
    print("rank\tplayer\tskill\tmu")
    for i in range(5):
        pnum = list(leaderboard.keys())[i]
        print(f"{i+1}\t{pnum}\t{comp.players[pnum].skill:.2f}\t{leaderboard[i]:.0f}")
    print("-------------")
```

</div>
<div class="column-right">

# 日本語訳

```python
for comp in C:
    scores = {p: v.rating.mu for p,v in comp.players.items()}
    leaderboard = {k:v for k,v in sorted(scores.items(), key=lambda item: item[1], reverse = True)}
    print("リーダーボード")
    print("rank\tplayer\tskill\tmu")
    for i in range(5):
        pnum = list(leaderboard.keys())[i]
        print(f"{i+1}\t{pnum}\t{comp.players[pnum].skill:.2f}\t{leaderboard[i]:.0f}")
    print("-------------")
```

</div>
</details>

In [ ]:
for comp in C:
    scores = {p: v.rating.mu for p,v in comp.players.items()}
    leaderboard = {k:v for k,v in sorted(scores.items(), key=lambda item: item[1], reverse = True)}
    print("リーダーボード")
    print("rank\tplayer\tskill\tmu")
    for i in range(5):
        pnum = list(leaderboard.keys())[i]
        print(f"{i+1}\t{pnum}\t{comp.players[pnum].skill:.2f}\t{leaderboard[i]:.0f}")
    print("-------------")